<a href="https://colab.research.google.com/github/hmghaly/km/blob/main/b2web.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/')
cwd="/content/drive/MyDrive/KM"
os.chdir(cwd)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


#1- Get the crawl data for dmain and country
save it to the corresponding directory

In [ ]:
#1- First, run common crawl query on the most recent crawl, for the domain required
#Find indexes here https://index.commoncrawl.org/, identify which crawl index to use
#identify crawl queries e.g. ".net.au" , ".com.nz" etc
#use the script utils/cdx-index-client.py to get the number of pages per each crawl
import os, sys, gzip, shutil
from subprocess import PIPE, Popen
# import gzip, os, sys, shutil
# import shutil
#!ls
sys.path.append("utils")
from utils.extraction_utils import *


crawl_version="CC-MAIN-2021-31"  #crawl indexes of July 21 - from https://index.commoncrawl.org/
crawl_version="CC-MAIN-2021-43" #October 2021

run_dir="nov21_au" #name of the current run (we use the month at which we performed the run, not necessarily of the index)
run_domains=["edu.au","com.au","net.au","gov.au","org.au"] #domains of interest in this run

# run_dir="nov21_nz" #name of the current run (we use the month at which we performed the run, not necessarily of the index)
# run_domains=["ac.nz","co.nz","net.nz","org.nz","govt.nz"] #domains of interest in this run
#run_domains=["org.nz"]

crawl_dir="crawl" #main crawl directory - don't change
urls_dir="urls" #<<<<<< url subdirectory - don't change

!chmod +x utils/cdx-index-client.py

def run_command(command0):
  with Popen(command0, stdout=PIPE, stderr=None, shell=True) as process:
      output = process.communicate()[0].decode("utf-8")
      print(output.strip("\n\r\t"))
 
for domain in run_domains:  
#for domain in ["org.au"]:
  full_dir_path=os.path.join(crawl_dir,run_dir,domain,urls_dir) #putting the crawled data into the url subdir
  if not os.path.exists(full_dir_path): os.makedirs(full_dir_path)

  print("working on domain", domain)
  print("number of pages")
  npages_command="utils/cdx-index-client.py -c %s *.%s --show-num-pages"%(crawl_version,domain)
  crawl_command="utils/cdx-index-client.py -c %s *.%s --fl url -z -d %s"%(crawl_version,domain,full_dir_path)
  run_command(npages_command)
  print("now crawling")
  run_command(crawl_command)
  print("---------")

print(">>>> Finished crawling from common crawl, organzing data now")

#=================================================================================#
#2- Then extract the list of urls from the results of the query (stored in gzip files) into text files 
main_dir_path=os.path.join(crawl_dir,run_dir)
counter_dict={}

for cur_dir in os.listdir(main_dir_path):
  #full_dpath=os.path.join(main_dir_path,cur_dir)
  full_dpath=os.path.join(main_dir_path,cur_dir,urls_dir)
  if not os.path.isdir(full_dpath): continue
  url_counter=0
  print(full_dpath)
  out_txt_fname=cur_dir+".txt"
  out_txt_fpath=os.path.join(main_dir_path,out_txt_fname)
  print("output file:", out_txt_fpath)
  list_fopen=open(out_txt_fpath,"w")
  files=os.listdir(full_dpath)
  print(len(files))
  for i,fname in enumerate(files):
    used_urls=[]
    if i%100==0: print(cur_dir, i, "out of", len(files))
    fpath=os.path.join(full_dpath,fname)
    tmp_fopen=open(fpath)
    for tf in tmp_fopen:
      cur_url=tf.split()[-1].strip('"{}')
      if cur_url.split(".")[-1].lower() in ["pdf","png","js","css","jpg"]: continue
      if cur_url.endswith("robots.txt") or cur_url.endswith("wp-admin"): continue
      if "ezproxy" in cur_url: continue
      bare_url=get_bare_url(cur_url)
      # print(bare_url)
      if bare_url in used_urls: continue
      used_urls.append(bare_url)
      if "com" in cur_dir and len(bare_url.split("."))>3: continue #skip subdomains for .com websites
      if "net" in cur_dir and len(bare_url.split("."))>3: continue #skip subdomains for .net websites
      new_url=cur_url.split("//")[0]+"//"+bare_url  
      url_counter+=1

      list_fopen.write(new_url+"\n")
  list_fopen.close()
  counter_dict[cur_dir]=url_counter

for key,val in counter_dict.items():
  print(key, val)

print(">>>> finished extracting URLs from crawl files")

#===============================================================================#
#Now combining all the text files with url lists into one big list
main_dir_path=os.path.join(crawl_dir,run_dir)
txt_files=[os.path.join(main_dir_path,f0) for f0 in os.listdir(main_dir_path) if f0.endswith(".txt")]
out_fpath=os.path.join(main_dir_path,"all_urls.txt")
print(txt_files)
print(out_fpath)
with open(out_fpath,'wb') as wfd:
    for f in txt_files:
        with open(f,'rb') as fd:
            shutil.copyfileobj(fd, wfd)

print(">>>> finished combining the extracted URLs from into the cached file:", out_fpath)
# !cat "edu.au.txt" "com.au.txt" "gov.au.txt" "org.au.txt" > "all_au_urls.txt"


working on domain edu.au
number of pages
181
now crawling

---------
working on domain com.au
number of pages
2257
now crawling

---------
working on domain net.au
number of pages
61
now crawling

---------
working on domain gov.au
number of pages
183
now crawling

---------
working on domain org.au
number of pages
162
now crawling

---------
>>>> Finished crawling from common crawl, organzing data now
crawl/nov21_au/edu.au/urls
output file: crawl/nov21_au/edu.au.txt
181
edu.au 0 out of 181
edu.au 100 out of 181
crawl/nov21_au/com.au/urls
output file: crawl/nov21_au/com.au.txt
2257
com.au 0 out of 2257
com.au 100 out of 2257
com.au 200 out of 2257
com.au 300 out of 2257
com.au 400 out of 2257
com.au 500 out of 2257
com.au 600 out of 2257
com.au 700 out of 2257
com.au 800 out of 2257
com.au 900 out of 2257
com.au 1000 out of 2257
com.au 1100 out of 2257
com.au 1200 out of 2257
com.au 1300 out of 2257
com.au 1400 out of 2257
com.au 1500 out of 2257
com.au 1600 out of 2257
com.au 1700 out

#2-Cache the crawled websites

In [ ]:
#Now run the caching script on the full data with 60 workers and a batch of 10000 - we need to get the script to accept arguments
!chmod +x utils/caching_utils.py
#!utils/caching_utils.py "crawl/oct21_nz_gov/all_urls.txt" "crawl/oct21_nz_gov/cached_all.txt" #with 10000 batch - 60 workers
#!utils/caching_utils.py "crawl/oct21_au/all_urls.txt" "crawl/oct21_au/cached_all.txt"
#!utils/caching_utils.py "crawl/oct21_nz/all_urls.txt" "crawl/oct21_nz/cached_all.txt"
#!utils/caching_utils.py "crawl/nov21_nz/all_urls.txt" "crawl/nov21_nz/cached_all.txt"
!utils/caching_utils.py "crawl/nov21_au/all_urls.txt" "crawl/nov21_au/cached_all3.txt"

processing URL list:  crawl/nov21_au/all_urls.txt
Output cached file:  crawl/nov21_au/cached_all3.txt
number of urls to process: 551904
start_i 0 end_i 551904 n_urls 551904
current i 0 time: 0.0
current i 10000 time: 528.05
current i 20000 time: 604.61
current i 30000 time: 649.97
current i 40000 time: 663.03
current i 50000 time: 635.83
current i 60000 time: 676.92
current i 70000 time: 662.99
current i 80000 time: 653.2
current i 90000 time: 662.59
current i 100000 time: 653.91
current i 110000 time: 624.47
current i 120000 time: 658.85
current i 130000 time: 665.96
current i 140000 time: 652.46
current i 150000 time: 676.84
current i 160000 time: 642.59
current i 170000 time: 655.22
current i 180000 time: 647.88
current i 190000 time: 656.32
current i 200000 time: 642.85
current i 210000 time: 644.92
current i 220000 time: 651.72
current i 230000 time: 657.37
current i 240000 time: 639.27
current i 250000 time: 623.26
current i 260000 time: 667.55
current i 270000 time: 621.81
curre

In [ ]:
!wc -l crawl/nov21_au/cached_all3.txt
!tail crawl/nov21_au/cached_all3.txt

# !wc -l crawl/nov21_nz/cached_all.txt
# !tail crawl/nov21_nz/cached_all.txt

# !wc -l crawl/aug21/all_urls.txt
# !tail crawl/aug21/all_urls.txt
# #357290 crawl/aug21/cached_all.txt
# !wc -l crawl/aug21/classified_all.txt
# !tail crawl/aug21/classified_all.txt

551290 crawl/nov21_au/cached_all3.txt
https://catholic.org.au<br>{"title": "Home - Catholic Church in Australia", "description": ""}<br>Home - Catholic Church in Australia<br>Search Search News Media Centre<br>×<br>Media Blog<br>Latest News<br>Fr Danny Meagher named Auxiliary Bishop of Sydney New Popular<br>Pope Francis appoints new auxiliary bishop for Brisbane Popular<br>Church shores up ability to support victims, survivors Popular<br>twitter Australian Catholic Bishops Conference @ACBC1 ACBC1 Today is the Solemnity of Our Lord Jesus Christ, King of the Universe, also known as the Feast of Christ the King.… https://t.co/iPl8BKAjm2 Nov 21 • reply • retweet • favorite ACBC1 Pope Francis has this evening appointed Fr Danny Meagher an auxiliary bishop for the Archdiocese of Sydney. Fr Meag… https://t.co/NJzobXGjdV Nov 18 • reply • retweet • favorite ACBC1 The local consultation phase for the 2023 Synod of Bishops has been extended in Australia by more than two months,… https://t.co/oraY

#3- Process the classification structure

In [ ]:
#We need to run this every time we update the classification structure
from utils.classification_utils import *
#xls_fpath="2021-10-16.xlsx"
xls_fpath="2021-10-27.xlsx"
xls_fpath="https://docs.google.com/spreadsheets/d/e/2PACX-1vRrCbem3cnMO6cUGGhwMft-j2rZpd08aoQi-2l55HDJjOwKaATiMJOmXy6T3WYcpQ/pub?output=xlsx"
out_fpath="data_dict7.json"
cur_structure_obj=cat_struct(xls_fpath)
cur_structure_obj.save(out_fpath)
print("processed classifiecation sheet, json output saved at:",out_fpath)


processed classifiecation sheet, json output saved at: data_dict7.json


In [ ]:
import json
fopen=open("data_dict3.json")
json_dict=json.loads(fopen.read())
fopen.close()
print(json_dict["child_dict"])

{'accounting': [['accounting-services', 'Accounting & Bookkeeping']], 'animal-farming': [['animal-nutrition-health-products', 'Animal Nutrition & Health'], ['livestock-farming', 'Livestock Farming'], ['animal-transport-handling', 'Animal Transport & Handling'], ['abattoirs', 'Abattoirs'], ['seafood-farming-supply', 'Seafood Farming & Supply'], ['poultry-farming', 'Poultry Farming'], ['animal-veterinary-services', 'Veterinary Services']], 'audit-compliance': [['asset-property-inspection-verification', 'Asset & Property Inspection'], ['environmental-consulting-audit-services', 'Environmental Consulting'], ['food-beverages-audit-compliance-related-services', 'Food Safety & Compliance'], ['safety-audit-compliance-services', 'Workplace Safety'], ['financial-audit-compliance-services', 'Financial Audit Services'], ['fraud-incident-investigation-services', 'Fraud Investigation']], 'business-communication': [['translation-interpreting-services', 'Translation & Interpreting'], ['printing-publis

#4- Now do the classification

In [3]:
#Run the classification script - roll back old function to create vecor
!chmod +x utils/classification_utils.py
#First agument: Excel file with the categories, second: cached file, third: output classification file
#!utils/classification_utils.py "290721.xlsx" "crawl/aug21/cached_all.txt" "crawl/aug21/classified_all5.txt" 
# "crawl/aug21/classified_all5.txt"
#!utils/classification_utils.py "290721.xlsx" "crawl/oct21_nz_gov/cached_all.txt" "crawl/oct21_nz_gov/classified_all.txt"
#! utils/classification_utils.py "crawl" "oct21_au" "cached_all.txt" "classified_all.txt" "14october21.xlsx" "data_dict_new.json"
#! utils/classification_utils.py "crawl" "oct21_au" "cached_all.txt" "classified_all.txt" "data_dict.json"
#! utils/classification_utils.py "crawl" "nov21_nz" "cached_all.txt" "classified_all.txt" "data_dict.json"
! utils/classification_utils.py "crawl" "nov21_au" "cached_all3.txt" "classified_all3.txt" "data_dict.json"
print("finished classification")

started to load model
['utils/classification_utils.py', 'crawl', 'nov21_au', 'cached_all3.txt', 'classified_all3.txt', 'data_dict.json']
cat_json_fpath data_dict.json
cached_fpath crawl/nov21_au/cached_all3.txt
classified_fpath crawl/nov21_au/classified_all3.txt
loaded categorization structure, converted to dictionary
[==================================================] 100.0% 1662.8/1662.8MB downloaded
tcmalloc: large alloc 3600007168 bytes == 0x55ae2bcb2000 @  0x7f6336726001 0x7f63331b054f 0x7f6333200b58 0x7f6333204b17 0x7f63332a3203 0x55ae28522544 0x55ae28522240 0x55ae28596627 0x55ae28590ced 0x55ae28523bda 0x55ae28592737 0x55ae285909ee 0x55ae28523bda 0x55ae28592737 0x55ae28523afa 0x55ae28595d00 0x55ae285909ee 0x55ae28523bda 0x55ae28595d00 0x55ae285909ee 0x55ae285906f3 0x55ae2865a4c2 0x55ae2865a83d 0x55ae2865a6e6 0x55ae28632163 0x55ae28631e0c 0x7f633550ebf7 0x55ae28631cea
loaded WV model 266.3140616416931
loaded categorization vector -4.076957702636719e-05
utils/classification_utils.

#OLD - Final grouping, sorting and analysis of the classified data

In [ ]:
#now analyzing the classified websites
#let's split the data classified into com/gov/edu/org directories
#now analyzing the classified websites

import json, os
from itertools import groupby
from utils.classification_utils import *
from utils.extraction_utils import *


# country_code="nz"
# run_dir="oct21_nz_gov"

country_code="au"
run_dir="oct21_au"
relevance_threshold=0.6 #exclude websites with relevance scoe less than 0.5

cat_json_fpath="data_dict.json"
classified_fname="classified_all.txt"

only_top_category=False #select only one category per website
n_top=3 #number of top categories


out_dir="classified"
crawl_dir="crawl"

#Load the json dict to get which domains to include for each category
json_fopen=open(cat_json_fpath)
cat_struct_obj=json.load(json_fopen)
json_fopen.close()
cat_domains_dict=cat_struct_obj["cat_domains_dict"]

classified_fpath=os.path.join(crawl_dir,run_dir,classified_fname)
out_dir_path=os.path.join(crawl_dir,run_dir,out_dir)
if not os.path.exists(out_dir_path): os.makedirs(out_dir_path)


classified_fopen=open(classified_fpath)
info_dict={}
all_pred_list=[]
used_urls=[]
for i_,f in enumerate(classified_fopen):
  #if i_<120000: continue
  #if i_>120000: break
  if i_%50000==0: print(i_)
  f_split=f.strip("\n\r\t").split("\t")
  if len(f_split)!=2: continue
  url,info_str=f_split
  url=url.split("<br>")[0]
  bare_url=get_bare_url(url)
  url_type=bare_url.split(".")[-2].lower()
  if url_type=="net": url_type="com" #<<<<<<<<<<<<<<<<<<<<< COMBINING .net with .com
  if url_type=="co": url_type="com" #<<<<<<<<<<<<<<<<<<<<< .co > .com
  if url_type=="govt": url_type="gov" #<<<<<<<<<<<<<<<<<<<<< .govt > .gov
  # if bare_url in used_urls: continue
  # used_urls.append(bare_url)
  if url_type=="com" and len(bare_url.split("."))>3: continue

  local_info_dict=json.loads(info_str)
  info_dict[bare_url]=local_info_dict
  top_preds=local_info_dict["top_preds"]
  if not top_preds: continue


  if only_top_category: all_pred_list.append((top_preds[0][0],url,top_preds[0][1]))
  else:
    for tp in top_preds[:n_top]:
      #print(tp[0],tp[1])
      all_pred_list.append((url_type,tp[0],url,tp[1]))

classified_fopen.close()

all_pred_list.sort(key=lambda x:x[0])
grouped=[(key,[v[1:] for v in group]) for key,group in groupby(all_pred_list,lambda x:x[0])]
for dom, grp in grouped: #dom is the domain name - com/gov/edu/org
  print(dom, len(grp))
  grp.sort(key=lambda x:x[0])
  grouped2=[(key,[v[1:] for v in group]) for key,group in groupby(grp,lambda x:x[0])]
  subdir_path=os.path.join(out_dir_path,dom) #for .com .gov ... etc directories under the main directory
  if not os.path.exists(subdir_path): os.makedirs(subdir_path)
  for cat_id, grp2 in grouped2:
    applicable_domains=cat_domains_dict.get(cat_id,[])
    is_applicable=dom in applicable_domains
    if not is_applicable: continue
    print(">>>> applicable domains:", applicable_domains, "actual domain:",dom, "is applicable:",is_applicable)
    cat_fpath=os.path.join(subdir_path,cat_id+".txt")
    cat_fopen=open(cat_fpath,"w")
    used_urls=[]
    grp2.sort(key=lambda x:-x[-1])
    print(cat_id, len(grp2))
    for g2 in grp2:
      url0,score0=g2
      if score0<relevance_threshold: continue
      bare_url0=get_bare_url(url0)
      # if bare_url0 in used_urls: continue
      # used_urls.append(bare_url0)
      local_info_obj={}
      cur_info=info_dict[bare_url0]
      top_words=cur_info["top_words"]

      top_preds=cur_info["top_preds"]
      #just_preds=["%s:%s"%(v[0],v[1]) for v in top_preds]
      just_preds=[(v[0],v[1]) for v in top_preds]
      #print(top_preds)
      cur_cat_words=[v[-1] for v in top_preds if v[0]==cat_id]
      if cur_cat_words: cur_cat_words=cur_cat_words[0] 
      local_info_obj["preds"]=just_preds
      local_info_obj["top_words"]=top_words
      local_info_obj["cat_words"]=cur_cat_words
      local_info_json=json.dumps(local_info_obj)

      line="%s\t%s\t%s\n"%(url0,score0,local_info_json)
      cat_fopen.write(line)

  cat_fopen.close()
  print("==============")

print("finished creating classified files for each domain, now zipping")
#zip_fname="%s-%s1.zip"%(country_code,run_code)
zip_fname="%s-%s"%(country_code,run_dir)
zip_fpath=os.path.join(crawl_dir,run_dir,zip_fname)
# zip_command='zip -r %s %s'%(zip_fpath,out_dir_path)
# os.system(zip_command)
import shutil
shutil.make_archive(zip_fpath, 'zip', out_dir_path)

#!zip -r classified-au-aug21-2.zip classified2/

0
50000
100000
150000
200000
250000
300000
350000
400000
450000
com 1297344
>>>> applicable domains: ['com'] actual domain: com is applicable: True
3d-printing 9159
>>>> applicable domains: ['com', 'gov', 'edu', 'org'] actual domain: com is applicable: True
accounting-services 2927
>>>> applicable domains: ['com'] actual domain: com is applicable: True
advertising-services 9201
>>>> applicable domains: ['com'] actual domain: com is applicable: True
animal-nutrition-health-products 5297
>>>> applicable domains: ['com'] actual domain: com is applicable: True
animal-transport-handling 44216
>>>> applicable domains: ['com', 'org'] actual domain: com is applicable: True
animal-veterinary-services 5347
>>>> applicable domains: ['com', 'gov', 'edu', 'org'] actual domain: com is applicable: True
architecture-design-services 8003
>>>> applicable domains: ['com'] actual domain: com is applicable: True
artistic-design-services 14043
>>>> applicable domains: ['com'] actual domain: com is applicabl

'/content/drive/My Drive/KM/crawl/oct21_au/au-oct21_au.zip'

#5- New sorting/grouping stage
group all urls by their category - sort by relevence, filter by relevence threshold

In [4]:
#now analyzing the classified websites
#let's split the data classified into com/gov/edu/org directories
#now analyzing the classified websites

import json, os
from itertools import groupby
from utils.classification_utils import *
from utils.extraction_utils import *
from utils.file_utils import *


# country_code="nz"
# run_dir="oct21_nz_gov"

# country_code="au"
# run_dir="oct21_au"
# country_code="nz"
# #run_dir="oct21_nz"
# run_dir="nov21_nz"
country_code="au"
run_dir="nov21_au"

relevance_threshold=0.6 #exclude websites with relevance scoe less than 0.5
out_dir="classified_new"

cat_json_fpath="data_dict.json"
classified_fname="classified_all3.txt"

min_n_words=50 #minimum number of words in a page (to avoid pages with very little content)
# only_top_category=False #select only one category per website
# n_top=3 #number of top categories



crawl_dir="crawl"

#Load the json dict to get which domains to include for each category
json_fopen=open(cat_json_fpath)
cat_struct_obj=json.load(json_fopen)
json_fopen.close()
cat_domains_dict=cat_struct_obj["cat_domains_dict"] #what domain are applicable for each category

#now reading the classified file and creating the folder for the grouping of output
classified_fpath=os.path.join(crawl_dir,run_dir,classified_fname)
out_dir_path=os.path.join(crawl_dir,run_dir,out_dir)
if not os.path.exists(out_dir_path): os.makedirs(out_dir_path)


classified_fopen=open(classified_fpath)
info_dict={}
used_dict={} #to avoid duplication
all_pred_list=[]
used_urls=[]
for i_,f in enumerate(classified_fopen):
  #if i_<120000: continue
  #if i_>10: break
  if i_%50000==0: print(i_)
  f_split=f.strip("\n\r\t").split("\t")
  #print(f_split)
  if len(f_split)!=2: continue
  url,info_str=f_split
  url=url.split("<br>")[0]
  bare_url=get_bare_url(url)
  if used_dict.get(bare_url,False): continue #to avoid duplication
  used_dict[bare_url]=True
  dom=bare_url.split(".")[-2].lower() #url_type is the same as domain
  rev_url=reverse_url(url) #reversed URL for harmonization with manual results, to allow alphabetical sorting

  #identifying the type of domain, accounting for exceptions .co.uk  .govt.nz .. etc
  if dom=="net": dom="com" #<<<<<<<<<<<<<<<<<<<<< COMBINING .net with .com
  if dom=="co": dom="com" #<<<<<<<<<<<<<<<<<<<<< .co > .com
  if dom=="govt": dom="gov" #<<<<<<<<<<<<<<<<<<<<< .govt > .gov
  if dom=="ac": dom="edu" #<<<<<<<<<<<<<<<<<<<<< .ac > .edu
  if dom=="com" and len(bare_url.split("."))>3: continue #no subdomains for .com sites


  local_info_dict=json.loads(info_str)
  local_info_dict["url"]=url
  #local_info_dict["title"]=""
  #local_info_dict["description"]=""
  #local_info_dict["locations"]=[]
  

  top_preds=local_info_dict["top_preds"]
  if not top_preds: continue #exclude pages without predictions

  n_words=local_info_dict.get("n_words",0)
  if n_words<min_n_words: continue  #exclude pages with very little content

  #print(url, rev_url,url_type)
  has_preds=False #if this website has predictions or not
  for tp in top_preds:
    cat_id0,score0,keywords0=tp
    if score0<relevance_threshold: continue #filtering predictions below the threshold
    applicable_domains=cat_domains_dict.get(cat_id0,[]) #filtering by the applicable domains (e.g. only .com for this category)
    is_applicable=dom in applicable_domains
    if not is_applicable: continue

    all_pred_list.append((dom,cat_id0,rev_url,url,score0))
    has_preds=True
  if not has_preds: continue
  info_dict[rev_url]=local_info_dict #add the current info for the URL to the info_dict for all results

classified_fopen.close()
print(len(all_pred_list))

all_pred_list.sort(key=lambda x:x[0])
grouped=[(key,[v[1:] for v in group]) for key,group in groupby(all_pred_list,lambda x:x[0])]
for dom, grp in grouped: #dom is the domain name - com/gov/edu/org
  print(dom, len(grp))
  grp.sort(key=lambda x:x[0])
  grouped2=[(key,[v[1:] for v in group]) for key,group in groupby(grp,lambda x:x[0])]
  subdir_path=os.path.join(out_dir_path,dom) #for .com .gov ... etc directories under the main directory
  if not os.path.exists(subdir_path): os.makedirs(subdir_path)  
  for cat_id, grp2 in grouped2:
    grp2.sort(key=lambda x:-x[-1])
    print(dom,cat_id, len(grp2),grp2[0])
    cat_fpath=os.path.join(subdir_path,cat_id+".txt")
    cat_fopen=open(cat_fpath,"w") 
    for item in grp2:
      cur_line= item[0].ljust(99)+"\n"
      cat_fopen.write(cur_line)
    cat_fopen.close()  
    # for g2 in grouped2[:10]:
    #   print(g2)
    print("-------")

#Now saving the info dict into a text file
info_dict_fpath=os.path.join(out_dir_path,"info_dict.txt")
info_dict_fopen=open(info_dict_fpath,"w")
for k,v in info_dict.items():
  local_info_json=json.dumps(v)
  line="%s\t%s\n"%(k,local_info_json)
  info_dict_fopen.write(line)
info_dict_fopen.close()



  # for a in grp[:10]:
  #   print(a)
  # print("------")

#   grouped2=[(key,[v[1:] for v in group]) for key,group in groupby(grp,lambda x:x[0])]
#   subdir_path=os.path.join(out_dir_path,dom) #for .com .gov ... etc directories under the main directory
#   if not os.path.exists(subdir_path): os.makedirs(subdir_path)
#   for cat_id, grp2 in grouped2:
#     applicable_domains=cat_domains_dict.get(cat_id,[])
#     is_applicable=dom in applicable_domains
#     if not is_applicable: continue
#     print(">>>> applicable domains:", applicable_domains, "actual domain:",dom, "is applicable:",is_applicable)
#     cat_fpath=os.path.join(subdir_path,cat_id+".txt")
#     cat_fopen=open(cat_fpath,"w")
#     used_urls=[]
#     grp2.sort(key=lambda x:-x[-1])
#     print(cat_id, len(grp2))
#     for g2 in grp2:
#       url0,score0=g2
#       if score0<relevance_threshold: continue
#       bare_url0=get_bare_url(url0)
#       # if bare_url0 in used_urls: continue
#       # used_urls.append(bare_url0)
#       local_info_obj={}
#       cur_info=info_dict[bare_url0]
#       top_words=cur_info["top_words"]

#       top_preds=cur_info["top_preds"]
#       #just_preds=["%s:%s"%(v[0],v[1]) for v in top_preds]
#       just_preds=[(v[0],v[1]) for v in top_preds]
#       #print(top_preds)
#       cur_cat_words=[v[-1] for v in top_preds if v[0]==cat_id]
#       if cur_cat_words: cur_cat_words=cur_cat_words[0] 
#       local_info_obj["preds"]=just_preds
#       local_info_obj["top_words"]=top_words
#       local_info_obj["cat_words"]=cur_cat_words
#       local_info_json=json.dumps(local_info_obj)

#       line="%s\t%s\t%s\n"%(url0,score0,local_info_json)
#       cat_fopen.write(line)

#   cat_fopen.close()
#   print("==============")

print("finished creating classified files for each domain, now zipping")
zip_fname="%s-%s"%(country_code,run_dir)
zip_fpath=os.path.join(crawl_dir,run_dir,zip_fname)
import shutil
shutil.make_archive(zip_fpath, 'zip', out_dir_path)

0
50000
100000
150000
200000
250000
300000
350000
400000
450000
649808
com 639555
com 3d-printing 2699 ('au.com.solid-concepts', 'http://solid-concepts.com.au', 0.8131)
-------
com accounting-services 1850 ('au.com.jansaccountants', 'http://jansaccountants.com.au', 0.8263)
-------
com advertising-services 4762 ('au.com.virtualadvertising', 'http://virtualadvertising.com.au', 0.8449)
-------
com animal-nutrition-health-products 3172 ('au.com.iahp', 'https://iahp.com.au', 0.8416)
-------
com animal-transport-handling 6497 ('au.com.hannaskylivestock', 'https://hannaskylivestock.com.au', 0.8442)
-------
com animal-veterinary-services 1564 ('au.com.crookwellvet', 'http://crookwellvet.com.au', 0.8068)
-------
com architecture-design-services 5744 ('au.com.arquero', 'http://arquero.com.au', 0.8141)
-------
com artistic-design-services 4645 ('au.com.wendysart', 'https://wendysart.com.au', 0.861)
-------
com asset-property-inspection-verification 3044 ('au.com.drmay', 'http://drmay.com.au', 0.8

'/content/drive/My Drive/KM/crawl/nov21_au/au-nov21_au.zip'

#6- Compiling location data


In [ ]:
#Compiling the list of city information obtained from https://simplemaps.com/data/au-cities
import pandas as pd
import os, json

locactions_dir="locations"
if not os.path.exists(locactions_dir): os.mkdir(locactions_dir)  
fname="eg-vocab.xlsx"
#fname="https://docs.google.com/spreadsheets/d/e/2PACX-1vSADyPiyt5p8aRtMx90gUbpE4dxWhWrpoRY6YyYpoVnEAzmIj5N1kTkTrdcvsxkigUoBd3J9l3B6IN_/pub?output=xlsx"
fname='https://docs.google.com/spreadsheets/d/e/2PACX-1vQFuoa3Af_CqxZTyyFmzB6K0zbrObDMOrXr7rDykm1knTxYbClSIS1TRJJKCf4bhsD0q25omNKTo1iF/pub?output=xlsx'
#xls  = pd.ExcelFile(fname, sheet_name=None)
sheets_dict = pd.read_excel(fname, sheet_name=None,keep_default_na=False)
for sh_name,sh_obj in sheets_dict.items():
  print(sh_name)
  counter=0
  city_list=[]
  city_dict={}
  for row in sh_obj.iterrows():
    row_dict=row[1].to_dict()
    population=row_dict["population"]
    if population<20000: continue
    city_name=row_dict["city"]
    city_name=city_name.replace(",","-")
    admin_name=row_dict["admin_name"]
    coords=(row_dict["lat"],row_dict["long"])
    row_dict.pop('lat', None)
    row_dict.pop('long', None)
    row_dict["coords"]=coords
    city_id="%s, %s"%(city_name,admin_name)
    print(city_id)
    city_list.append(city_id)
    city_dict[city_id]=row_dict
    #print(row_dict)
    counter+=1
    #print(row_dict)
    #print("-------")
  country_city_dict={}
  country_city_dict["city_list"]=city_list
  country_city_dict["city_dict"]=city_dict
  json_fpath=os.path.join(locactions_dir,sh_name+".json")
  json_fopen=open(json_fpath,"w")
  json.dump(country_city_dict,json_fopen)
  json_fopen.close()


  print("total %s cities in %s"%(counter,sh_name))


#sheets_dict.keys()
# print(xls.sheet_names)
# for sh in xls.sheet_names:
#   cur_sheet=xls.

# for row in conversion_sheet.iterrows():
#   cur_obj={}
#   row_dict=row[1].to_dict()
#   print(row_dict)
#   print("-----")
  #latin2ar_list.append([str(row_dict["en"]),str(row_dict["ar"])])
#conversion_sheet=pd.read_excel(xls, 'conversion',keep_default_na=False)
#verb_sheet=pd.read_excel(xls, 'verbs',keep_default_na=False)
#print(conversion_sheet.keys())
#print(verb_sheet.keys())


au
Adelaide, South Australia
Albany, Western Australia
Albury, New South Wales
Alice Springs, Northern Territory
Armidale, New South Wales
Bacchus Marsh, Victoria
Ballarat, Victoria
Ballina, New South Wales
Bathurst, New South Wales
Bendigo, Victoria
Brisbane, Queensland
Bunbury, Western Australia
Bundaberg, Queensland
Busselton, Western Australia
Caboolture, Queensland
Cairns, Queensland
Caloundra, Queensland
Canberra, Australian Capital Territory
Cessnock, New South Wales
Coffs Harbour, New South Wales
Cranbourne, Victoria
Dandenong, Victoria
Darwin, Northern Territory
Devonport, Tasmania
Dubbo, New South Wales
Frankston, Victoria
Gawler, South Australia
Geelong, Victoria
Geraldton, Western Australia
Gladstone, Queensland
Gold Coast, Queensland
Goulburn, New South Wales
Griffith, New South Wales
Gympie, Queensland
Hervey Bay, Queensland
Hobart, Tasmania
Ipswich, Queensland
Kalgoorlie, Western Australia
Kwinana, Western Australia
Launceston, Tasmania
Lismore, New South Wales
Mackay, Q

#7- Compiling a word Vector from data

In [ ]:
import time, os, re
#from code_utils.general import *
from gensim.models import Word2Vec

def tok(txt):
  return re.findall("\w+",txt)

def get_sentences(fpath):
  fopen=open(fpath)
  tmp_sentences=[]
  sent_counter=0
  completed=False
  for i,site_content in enumerate(fopen):
    if completed: break
    content_split=site_content.split("<br>")
    #print(len(content_split), content_split[0])
    if len(content_split)<4: continue
    for line in content_split:
      line=line.strip("\n\r\t ")
      toks=tok(line.lower())
      if len(toks)<3: continue
      sent_counter+=1
      if sent_counter%10000==0: print(sent_counter)
      if sent_counter>2000000: 
        completed=True
        break
      tmp_sentences.append(toks)
      #yield toks
  fopen.close()
  return tmp_sentences

model_fname="b2web_au.model"

t0=time.time()
data_fpath="crawl/oct21_au/cached_all.txt"
t0=time.time()
sentences=get_sentences(data_fpath)
print("collected %s sentences"%len(sentences))
print("Now creating the WV model")
b2web_model = Word2Vec(sentences, size=100, min_count=5) #small model
t1=time.time()
print("collected sentences in %s seconds"%(t1-t0))
fopen.close()

b2web_model.save(model_fname)
print("saved model as: ",model_fname)

# t0=time.time()
# wv_model = Word2Vec(sentences, size=300, min_count=5)
# t1=time.time()
# print("trained model in %s seconds"%(t1-t0))


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
1390

In [ ]:
import gensim
b2web_model = gensim.models.Word2Vec.load("b2web_au.model")
b2web_model["about"]
b2web_model.most_similar("agriculture")

b2web_model.similar_by_vector

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  after removing the cwd from sys.path.


<bound method BaseWordEmbeddingsModel.similar_by_vector of <gensim.models.word2vec.Word2Vec object at 0x7ff60c8e4d90>>

In [ ]:
import json
fopen=open("data_dict3.json")
json_dict=json.loads(fopen.read())
fopen.close()
keyword_dict=json_dict["keyword_dict"]
vector_dict={}
for cat0,keywords0 in keyword_dict.items():
  if keywords0==[]: continue
  #print(cat0, keywords0)
  tmp_vec0,tmp_wd_vec_dict=get_words_vector(keywords0,b2web_model,excluded_words=[])
  vector_dict[cat0]=tmp_vec0

from utils.classification_utils import *
query_sent="juice milk"
query_words=re.findall("\w+",query_sent.lower())
query_vec,query_wd_vector_dict=get_words_vector(query_words,b2web_model,excluded_words=[])
cat_relevance_list=[]
for cat0,vec0 in vector_dict.items():
  sim_score=cos_sim(query_vec,vec0)
  cat_relevance_list.append((cat0,sim_score))
  #print(cat0,sim_score)
cat_relevance_list.sort(key=lambda x:-x[-1])
for a in cat_relevance_list[:5]:
  print(a)


#print()


('grains-farming-supply', 0.7443616986274719)
('fruit-farming-supply', 0.7210757732391357)
('vegetables-farming-supply', 0.6994040012359619)
('food-beverages-online-retail-wholesale', 0.6787687540054321)
('herbs-seeds-and-organic-farming', 0.6276875138282776)


#Converting the gensim file into h5py file

In [ ]:
import h5py
import gensim
b2web_model = gensim.models.Word2Vec.load("b2web_au.model")
vocab_len = len(b2web_model.wv.vocab)
print(vocab_len)
h5_fopen = h5py.File("au_wv.hdf5", "w")

for i,word in enumerate(b2web_model.wv.vocab):
  #if i>100: break
  if i%10000==0: print(i)
  #cur_vec=b2web_model[word]
  cur_vec=b2web_model.wv.get_vector(word)
  #print(i,word,cur_vec.shape)
  dset = h5_fopen.create_dataset(word, data=cur_vec, compression="gzip")
  # dset = f.create_dataset("init", data=arr)
  # dset = f.create_dataset("zipped", (100, 100), compression="gzip")


h5_fopen.close()

91199
0
10000
20000
30000
40000
50000
60000
70000
80000
90000


In [ ]:
import time
import numpy as np
h5_fopen = h5py.File("au_wv.hdf5", "r")
t0=time.time()
out=h5_fopen["about"]
array=np.array(out)
print(array)
#out.numpy

t1=time.time()
print(out,t1-t0)
#print(len(h5_fopen.keys()))
h5_fopen.close()

#h5_fopen.close()

[-0.6273137  -2.726678   -3.638985   -2.5158987  -3.4097922   0.4896717
 -3.2217185   2.6280441   0.29165864 -0.5793313   0.9262441   0.75802183
 -0.0890814   0.55190754 -0.03783246  0.8031593   0.12457802 -0.41907343
 -2.637877   -0.88862634 -5.811431    1.762541    2.4001756  -2.5652215
 -0.30599412  0.2609674   0.32453522 -0.96270144 -0.16386965 -1.5500721
  0.48222178 -2.0455775  -3.3646753   1.4691088   1.8308747  -3.2601526
 -0.6451963   0.4681361  -0.7433473   1.7979311   1.5575551   0.30023623
 -1.0259432  -0.04932436 -0.2582866  -0.5648833  -0.14092587  0.23305474
 -1.0700854   0.07542213 -0.50416     0.22070359  0.69965506  1.6025642
 -0.06584207 -3.4080749   1.4394017  -1.4026273   1.1786438  -3.1787384
 -1.1070263   3.6111429   0.3762302   0.5072073  -4.52797    -2.115355
  1.6564044  -1.8377428   0.38359982  0.62724096  2.662227   -1.169268
  1.4340005  -0.53743273 -2.6414363  -1.7475449  -0.2855417  -1.3947943
 -0.7862146   0.6844968  -2.4239964  -0.7684873  -0.7418077  -

In [ ]:
import re
from collections import Counter
import time
import numpy as np

def get_words_vector(words,wv_model,excluded_words=[]):
  wd_counter=Counter(words)
  flag=False
  total_count0=0
  wd_vector_dict={}
  for wd0,wd0_n in wd_counter.items():
    if wd0 in excluded_words: continue
    try: cur_vec0=wv_model[wd0]
    except: continue
    wd_vector_dict[wd0]=cur_vec0
    weighted_vec0=cur_vec0*wd0_n
    if flag==False: total_vec0=weighted_vec0
    else: total_vec0+=weighted_vec0
    flag=True
    total_count0+=wd0_n
  if total_count0==0: return [],{}
  avg_vec=total_vec0/total_count0
  return avg_vec,wd_vector_dict

def get_h5_words_vec(words0,h5_obj0):
  tmp_dict={} #get a temporary dict from the found words in the opened h5 file
  for w0 in words0:
    out=h5_obj0.get(w0)
    if out!=None: tmp_dict[w0]=np.array(out)
  cur_overall_vec,cur_wd_vec=get_words_vector(words,tmp_dict,excluded_words=[])
  return cur_overall_vec,cur_wd_vec



sent="agricultural tractors"
words=re.findall("\w+",sent.lower())
h5_fopen = h5py.File("au_wv.hdf5", "r")

t0=time.time()
cur_vec0,wd_vec0=get_h5_words_vec(words,h5_fopen)

# tmp_dict={}
# for w0 in words:
#   out=h5_fopen.get(w0)
#   if out!=None: tmp_dict[w0]=np.array(out)
#   #out = np.array(out) h5_fopen.get(w0).value
  

# print(tmp_dict)
# cur_overall_vec,cur_wd_vec=get_words_vector(words,tmp_dict,excluded_words=[])
t1=time.time()
h5_fopen.close()
print(cur_vec0,t1-t0)

[ 0.5577053   0.45792735  1.0450709   0.562682   -0.12847863  2.0843368
 -0.31042415 -0.81095713 -0.5664619  -1.2944973   0.4357319  -0.09270044
  0.9631035  -0.6987387   0.06128725 -0.8378145  -0.2110319   0.251157
 -0.6325569  -0.27878588  0.6457068  -0.4853387   0.543125    0.27706572
  0.01710689 -0.64405286 -0.11107025 -0.2164903   0.23709536  1.3793913
 -0.3402099  -0.61247164 -1.9594917  -0.78364134 -0.31314313 -0.52277637
  0.03695814 -0.65011024 -0.41623294  0.13565418 -1.440279    1.186044
 -0.59626085 -0.5987468  -0.7711021  -0.5752859  -1.1747632  -0.13529378
  0.68363434  0.76355505 -0.49488485 -0.87701285 -1.5275435  -0.01794878
 -0.9953402  -0.28605765  1.1922851   0.29251164  0.33632872  1.8044391
  0.54500306  1.1654642  -0.75974137 -0.40648264  0.92965895 -0.02688909
  1.1286027  -1.1028028   1.6797354  -0.02251232  0.89712805  0.8334291
 -0.8085667  -0.5107296   0.09774169 -0.5641119   0.30860096  0.5480943
  0.29711986 -0.20176092 -0.19783856  0.3183217  -0.05367745

In [ ]:
!chmod +x utils/h5py_utils.py
!python utils/h5py_utils.py

[ 0.5577053   0.45792735  1.0450709   0.562682   -0.12847863  2.0843368
 -0.31042415 -0.81095713 -0.5664619  -1.2944973   0.4357319  -0.09270044
  0.9631035  -0.6987387   0.06128725 -0.8378145  -0.2110319   0.251157
 -0.6325569  -0.27878588  0.6457068  -0.4853387   0.543125    0.27706572
  0.01710689 -0.64405286 -0.11107025 -0.2164903   0.23709536  1.3793913
 -0.3402099  -0.61247164 -1.9594917  -0.78364134 -0.31314313 -0.52277637
  0.03695814 -0.65011024 -0.41623294  0.13565418 -1.440279    1.186044
 -0.59626085 -0.5987468  -0.7711021  -0.5752859  -1.1747632  -0.13529378
  0.68363434  0.76355505 -0.49488485 -0.87701285 -1.5275435  -0.01794878
 -0.9953402  -0.28605765  1.1922851   0.29251164  0.33632872  1.8044391
  0.54500306  1.1654642  -0.75974137 -0.40648264  0.92965895 -0.02688909
  1.1286027  -1.1028028   1.6797354  -0.02251232  0.89712805  0.8334291
 -0.8085667  -0.5107296   0.09774169 -0.5641119   0.30860096  0.5480943
  0.29711986 -0.20176092 -0.19783856  0.3183217  -0.05367745

In [ ]:
!head crawl/oct21_au/classified_new/info_dict.txt

au.edu.uow.research-impact	{"top_words": ["uow", "societal", "impact", "research", "global", "environmental", "regional", "studies", "case", "stories"], "n_words": 548, "top_preds": [["investment-finance-advisory-services", 0.6335, ["business", "financial", "management", "economic", "global"]], ["market-research-and-analysis-services", 0.6083, ["research", "data", "industry", "global", "economic"]], ["safety-audit-compliance-services", 0.6031, ["safety", "environmental", "health", "management", "security"]], ["financial-audit-compliance-services", 0.5763, ["financial", "governance", "business", "management", "ethics"]], ["legal-consulting-advisory-services", 0.5713, ["law", "expert", "business", "policy", "case"]]], "url": "https://research-impact.uow.edu.au"}
au.edu.uow.unishop	{"top_words": ["uow", "unishop", "search", "engineering", "merchandise", "home", "products", "textbooks", "sciences", "arts"], "n_words": 375, "top_preds": [["it-hardware-retail", 0.6556, ["computer", "computin

#Other Old stuff

In [ ]:
from utils.classification_utils import *
xls_fname="14october21.xlsx"
categories_obj=cat_struct(xls_fname)
print(categories_obj.parent_list)
print(len(categories_obj.parent_list))
for k,v in categories_obj.child_dict.items():
  print(k,v)
print(len(categories_obj.child_dict))
categories_obj.keyword_dict

In [ ]:
!chmod +x utils/classification_utils.py
! utils/classification_utils.py crawl/ oct21_au "cached_all.txt" "classified_all.txt" "14october21.xlsx" "data_dict_new.json"

started to load model
['utils/classification_utils.py', 'crawl/', 'oct21_au', 'cached_all.txt', 'classified_all.txt', '14october21.xlsx', 'data_dict_new123.json']
xl_fname 290721.xlsx
cached_fpath crawl/oct21_au/cached_all.txt
classified_fpath crawl/oct21_au/classified_all.txt
loaded categorization structure, converted to json


In [ ]:
#Step 1 - load the Word2Vec model
# #from gensim.models import Doc2Vec
import time
import gensim.downloader as api
t0=time.time()
# #wv = api.load('word2vec-google-news-300', return_path=True)
model = api.load("word2vec-google-news-300")  # download the model and return as object ready for use
t1=time.time()
elapsed=t1-t0
print("Time for loading the model:", elapsed)
# word_vectors = model.wv #load the vectors from the model
# model.save("google-news.wv")
#Now loading the model
# from gensim.models import KeyedVectors
# wv_model=KeyedVectors.load("google-news.wv",mmap='r')


Time for loading the model: 135.4795446395874


In [ ]:
!head -10 crawl/aug21/classified_all5.txt

http://web.med.unsw.edu.au	{"top_words": ["webmed", "unsw", "service", "this", "page", "longer", "active", "the", "sites", "hosted"], "n_words": 56, "top_preds": [["web-development-services", 0.6649, ["web", "page", "sites", "edu", "log"]], ["it-hardware-retail", 0.554, ["computing", "services", "web", "sites", "service"]], ["information-security", 0.5388, ["web", "computing", "services", "log", "sites"]], ["software-app-development-services", 0.5223, ["web", "computing", "amazon", "services", "service"]], ["it-systems-network-maintenance-services", 0.5061, ["services", "service", "support", "computing", "web"]]]}
http://staging.optometry.unsw.edu.au	{"top_words": ["unauthorized", "credentials", "this", "server", "could", "verify", "authorized", "access", "document", "requested"], "n_words": 41, "top_preds": [["it-hardware-retail", 0.5169, ["server", "browser", "password", "supply", "access"]], ["information-security", 0.5146, ["server", "password", "access", "browser", "verify"]], ["e

In [ ]:
!head -10 crawl/aug21/classified_all3.txt

http://web.med.unsw.edu.au	{"top_words": ["webmed", "unsw", "service", "this", "page", "longer", "active", "the", "sites", "hosted"], "n_words": 56, "top_preds": [["web-development-services", 0.6649, ["web", "page", "sites", "edu", "log"]], ["it-hardware-retail", 0.554, ["computing", "services", "web", "sites", "service"]], ["information-security", 0.5388, ["web", "computing", "services", "log", "sites"]], ["software-app-development-services", 0.5223, ["web", "computing", "amazon", "services", "service"]], ["it-systems-network-maintenance-services", 0.5061, ["services", "service", "support", "computing", "web"]]]}
http://staging.optometry.unsw.edu.au	{"top_words": ["unauthorized", "credentials", "this", "server", "could", "verify", "authorized", "access", "document", "requested"], "n_words": 41, "top_preds": [["it-hardware-retail", 0.5169, ["server", "browser", "password", "supply", "access"]], ["information-security", 0.5146, ["server", "password", "access", "browser", "verify"]], ["e

In [ ]:
from utils.classification_utils import *

class text_classification:
  def __init__(self,text_input,wv_model_input,category_vector_dict_input,excluded_words=[],pred_n=5,related_words_n=5):
    self.wv_model=wv_model_input
    self.category_vector_dict=category_vector_dict_input
    # tmp_text_split=text_input.split(" ")
    # self.url=tmp_text_split[0]
    self.text=text_input #" ".join(tmp_text_split[1:])
    self.excluded_words=excluded_words+noise_words
    self.words=[v.lower() for v in re.findall("\w+",self.text) if not v in self.excluded_words and not v[0].isdigit() and len(v)>2]
    self.text_vec,self.word_vec_dict=get_words_vector(self.words,self.wv_model,self.excluded_words)
    #print("self.words",self.words)
    #print(self.text_vec)
    self.preds=[]
    self.word_counts=Counter(self.words)
    self.top_preds=[] #top predictions with their values and associated words

    if self.text_vec==[]: return
    for cat,vec0 in self.category_vector_dict.items():
      similarity_val=cos_sim(vec0,self.text_vec)
      self.preds.append((cat,similarity_val))
      #print(cat,similarity_val)
    self.preds.sort(key=lambda x:-x[1])
    for pred_cat,pred_val in self.preds[:pred_n]:
      pred_vec=self.category_vector_dict[pred_cat]
      wd_sim_list=[]
      for wd0,vec0 in self.word_vec_dict.items():
        wd_cat_sim_val=cos_sim(pred_vec,vec0)
        wd_sim_list.append((wd0,wd_cat_sim_val))
      wd_sim_list.sort(key=lambda x:-x[1])
      related_words=[v[0] for v in wd_sim_list[:related_words_n]]
      self.top_preds.append((pred_cat,round(pred_val,4),related_words))


cur_wv_model=model
xl_fname='290721.xlsx'
cat_struct_obj=cat_struct(xl_fname)
cat_struct_obj.save("data_dict_new2.json")
cur_keyword_dict=cat_struct_obj.keyword_dict 
cur_cat_vector=create_cat_vector(cur_keyword_dict,cur_wv_model) 
print("finished processing and saving the classification structure") 

t0=time.time()
cached_fpath="crawl/aug21/cached_all.txt"
classified_fpath="crawl/aug21/classified_all4.txt"
fopen=open(cached_fpath)
for i,f in enumerate(fopen):
  line=f.strip("\n")
  content=" ".join(line.split("<br>")[1:])
  words=[v.lower() for v in re.findall("\w+",content) if not v in en_stop_words and not v[0].isdigit() and len(v)>2]
  print(words[:10])
  #word_vector,wd_vector_dict=get_words_vector(words,cur_wv_model,excluded_words=[])
  tc_obj=text_classification(content,cur_wv_model,cur_cat_vector,excluded_words=en_stop_words)
  print(tc_obj.top_preds)

  #print(word_vector)
  if i>10: break
fopen.close()

#classify_cached(cached_fpath,classified_fpath,cur_cat_vector,cur_wv_model)
#cur_cat_vector=create_cat_vector(kw_dict0,cur_wv_model)
t1=time.time()
elapsed=t1-t0
print("Time for classification:", elapsed)


finished processing and saving the classification structure
[]
[]
['not', 'found', 'not', 'found', 'not', 'found', 'the', 'requested', 'url', 'found']
[('it-systems-network-maintenance-services', 0.3757, ['server', 'handle', 'use', 'encountered', 'trying']), ('web-development-services', 0.3708, ['url', 'server', 'use', 'additionally', 'error']), ('software-app-development-services', 0.3526, ['server', 'url', 'use', 'additionally', 'error']), ('it-hardware-retail', 0.3483, ['server', 'use', 'url', 'additionally', 'handle']), ('safety-audit-compliance-services', 0.3455, ['use', 'found', 'handle', 'additionally', 'request'])]
['webmed', 'webmed', 'this', 'page', 'longer', 'active', 'the', 'sites', 'hosted', 'amazon']
[('web-development-services', 0.6649, ['web', 'page', 'sites', 'edu', 'log']), ('it-hardware-retail', 0.554, ['computing', 'services', 'web', 'sites', 'service']), ('information-security', 0.5388, ['web', 'computing', 'services', 'log', 'sites']), ('software-app-development-s

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


[('animal-transport-logistics', 0.5375, ['bird', 'highway', 'fuel', 'tom', 'bed']), ('web-development-services', 0.5045, ['web', 'sites', 'edu', 'darwin', 'network']), ('food-beverages-online-retail-wholesale', 0.5014, ['coffee', 'meals', 'meal', 'goods', 'juice']), ('hotels-accommodation', 0.4985, ['hotel', 'accommodation', 'hotels', 'rooms', 'room']), ('energy-consulting-services', 0.4932, ['compliant', 'report', 'fuel', 'electric', 'notification'])]
[]
[]
[]
[]
['emed', 'login', 'login', 'need', 'enter', 'zid', 'zpass', 'cannot', 'login', 'using']
[('web-development-services', 0.6857, ['web', 'internet', 'page', 'browsers', 'firefox']), ('software-app-development-services', 0.5841, ['browsers', 'firefox', 'web', 'code', 'internet']), ('information-security', 0.571, ['information', 'internet', 'web', 'password', 'browsers']), ('it-hardware-retail', 0.5487, ['browsers', 'internet', 'provider', 'web', 'password']), ('energy-consulting-services', 0.5273, ['guidelines', 'information', 'n

#testing to optimize the classification

In [ ]:
!chmod +x utils/cdx-index-client.py
!utils/cdx-index-client.py -c CC-MAIN-2021-39 *.edu.au --show-num-pages

2021-10-12 01:39:46,150: [INFO]: Getting Index From https://index.commoncrawl.org/CC-MAIN-2021-39-index
162


In [ ]:
crawl_version="CC-MAIN-2021-39"
domain="edu.au"
a="utils/cdx-index-client.py -c %s *.%s --show-num-pages"%(crawl_version,domain)
print(a)

utils/cdx-index-client.py -c CC-MAIN-2021-39 *.edu.au --show-num-pages


In [ ]:
from utils.classification_utils import *
from collections import Counter

cur_wv_model=model
def get_words_vector_new(words,wv_model,excluded_words=[]):
  wd_counter=Counter(words)
  flag=False
  total_count0=0
  wd_vector_dict={}
  for wd0,wd0_n in wd_counter.items():
    if wd0 in excluded_words: continue
    try: cur_vec0=wv_model[wd0]
    except: continue
    wd_vector_dict[wd0]=cur_vec0
    weighted_vec0=cur_vec0*wd0_n
    if flag==False: total_vec0=weighted_vec0
    else: total_vec0+=weighted_vec0
    flag=True
    total_count0+=wd0_n
  if total_count0==0: return [],{}
  avg_vec=total_vec0/total_count0
  return avg_vec,wd_vector_dict

cur_excluded_words=en_stop_words+noise_words

#text_vec,word_vec_dict=get_words_vector(self.words,self.wv_model,self.excluded_words)

t0=time.time()
cached_fpath="crawl/aug21/cached_all.txt"
classified_fpath="crawl/aug21/classified_all5.txt"
fopen=open(cached_fpath)
for i,f in enumerate(fopen):
  line=f.strip("\n")
  content=" ".join(line.split("<br>")[1:])
  words=[v.lower() for v in re.findall("\w+",content) if not v in en_stop_words and not v[0].isdigit() and len(v)>2]
  word_vector,wd_vector_dict=get_words_vector(words,cur_wv_model,excluded_words=cur_excluded_words)
  print(word_vector[:10])
  word_vector,wd_vector_dict=get_words_vector_new(words,cur_wv_model,excluded_words=cur_excluded_words)
  print(word_vector[:10])
  print("-----")  

  # tc_obj=text_classification(content,cur_wv_model,cur_cat_vector,excluded_words=en_stop_words)
  # print(tc_obj.top_preds)

  #print(word_vector)
  if i>100: break
fopen.close()
wd_counter=Counter(words)
#classify_cached(cached_fpath,classified_fpath,cur_cat_vector,cur_wv_model)
#cur_cat_vector=create_cat_vector(kw_dict0,cur_wv_model)
t1=time.time()
elapsed=t1-t0
print("Time for classification:", elapsed)

#Old method, 1000 items, Time for classification: 28.20668387413025
#New Method, 1000 items 2.6769816875457764

[]
[]
-----
[ 0.00885417  0.00110677 -0.02493083  0.03610408 -0.06189779 -0.02643433
  0.09690755  0.01086032  0.08762207  0.07668457]
[ 0.00885417  0.00110677 -0.02493083  0.03610408 -0.06189779 -0.02643433
  0.09690755  0.01086032  0.08762207  0.07668457]
-----
[ 0.0032848  -0.01595818  0.02336329  0.05948431 -0.06183277  0.06359031
  0.02067739 -0.06714977  0.0947765  -0.04185624]
[ 0.0032848  -0.01595818  0.02336329  0.05948431 -0.06183277  0.06359031
  0.02067739 -0.06714977  0.0947765  -0.04185624]
-----
[ 0.02748471 -0.03815569  0.02421061 -0.10731725 -0.09386045  0.07542783
  0.03511556  0.07301222  0.1397647  -0.07976423]
[ 0.02748471 -0.03815569  0.02421061 -0.10731725 -0.09386045  0.07542783
  0.03511556  0.07301222  0.1397647  -0.07976423]
-----
[ 0.11399078 -0.02883911  0.18024445  0.07711792 -0.0457077   0.01687622
  0.15126038 -0.19029236 -0.08466339 -0.01216507]
[ 0.11399078 -0.02883911  0.18024445  0.07711792 -0.0457077   0.01687622
  0.15126038 -0.19029236 -0.08466339

In [ ]:
flag=False
total_count=0
wd_vector_dict={}
for wd0,wd0_n in wd_counter.items():
  try: cur_vec0=cur_wv_model[wd0]
  except: continue
  wd_vector_dict[wd0]=cur_vec0
  #print(wd0,wd0_n)
  weighted_vec0=cur_vec0*wd0_n
  if flag==False: total_vec=weighted_vec0
  else: total_vec+=weighted_vec0
  flag=True
  total_count+=wd0_n
  #print("----")
avg_vec=total_vec/total_count
print(total_vec.shape)
print(total_vec[:10])
print(avg_vec[:10])

----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
----
(300,)
[ 0.7790375  -0.23046875  0.2920189   1.2869263  -1.4663086   0.1274414
  1.8898926  -1.9942017   0.2514038  -2.3354187 ]
[ 0.02996298 -0.00886418  0.0112315   0.04949716 -0.05639648  0.00490159
  0.07268818 -0.07670006  0.00966938 -0.0898238 ]


In [ ]:
#Get the classification structure and save it as json
from utils.classification_utils import *
xl_fname='290721.xlsx'
cat_struct_obj=cat_struct(xl_fname)
cat_struct_obj.save("data_dict_new.json")
#cat_struct_obj.data_dict

{'child_dict': {'accounting': [('accounting-services',
    'Accounting & Bookkeeping')],
  'audit-compliance': [('financial-audit-compliance-services', 'Financial'),
   ('fraud-incident-investigation-services', 'Fraud Investigation'),
   ('environmental-consulting-audit-services', 'Environmental'),
   ('safety-audit-compliance-services', 'Workplace Safety'),
   ('financial-audit-compliance-services', 'Audit & Compliance')],
  'banking-financial-investment-services': [('investment-finance-advisory-services',
    'Financial'),
   ('investment-finance-advisory-services', 'Financial Consulting'),
   ('investment-fund-management-services', 'Investment'),
   ('lending-services', 'Lending')],
  'clothing-products': [('workwear', 'Workwear')],
  'construction': [('paint-coating', 'Paints'),
   ('commercial-industrial-heating-cooling',
    'Commercial & Industrial Heating & Cooling'),
   ('commercial-construction-services', 'Commercial Construction'),
   ('residential-construction-renovation-se

In [ ]:
#Run the classification script
!chmod +x utils/classification_utils.py
!utils/classification_utils.py 
#Time for classification: 35624.586965084076 #without any optimization

started to load model
loaded model 9.649028062820435
loaded classification structure -4.935264587402344e-05
utils/classification_utils.py:100: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if self.text_vec==[]: return
5000 time: 294.91
10000 time: 280.02
15000 time: 285.3
20000 time: 596.35
25000 time: 335.51
35000 time: 650.1
40000 time: 321.09
45000 time: 323.5
50000 time: 308.05
55000 time: 322.46
60000 time: 336.36
65000 time: 327.67
70000 time: 321.09
75000 time: 329.97
85000 time: 661.38
90000 time: 323.59
95000 time: 325.59
100000 time: 331.92
110000 time: 632.56
115000 time: 315.42
130000 time: 974.08
135000 time: 334.42
140000 time: 311.64
145000 time: 313.16
150000 time: 326.29
155000 time: 324.31
160000 time: 333.73
165000 time: 321.03
170000 time: 312.93
175000 time: 311.12
180000 time: 313.71
185000 time: 309.91
190000 time: 335.14
195000 time: 318.36
200000 time: 314.7
205000 time: 317.4
210000 time: 330.71
215000 time: 325.9

In [ ]:
#Try to use pypy to make it faster
!pypy3 utils/classification_utils.py 

Traceback (most recent call last):
  File "utils/classification_utils.py", line 2, in <module>
    from gensim.models import KeyedVectors
ModuleNotFoundError: No module named 'gensim'


In [ ]:
import os
run_dir="aug21" #aug21_au <<<<<<<<<<
country_code="au"
!zip -r classified-au-aug21-2.zip classified2/

  adding: classified2/ (stored 0%)
  adding: classified2/au/ (stored 0%)
  adding: classified2/au/3d-printing.txt (deflated 37%)
  adding: classified2/au/animal-transport-logistics.txt (deflated 39%)
  adding: classified2/au/manufacture-of-metallic-components-structures.txt (deflated 37%)
  adding: classified2/com/ (stored 0%)
  adding: classified2/com/3d-printing.txt (deflated 86%)
  adding: classified2/com/accounting-services.txt (deflated 90%)
  adding: classified2/com/advertising-services.txt (deflated 88%)
  adding: classified2/com/animal-nutrition-health-products.txt (deflated 87%)
  adding: classified2/com/animal-transport-logistics.txt (deflated 86%)
  adding: classified2/com/animal-veterinary-services.txt (deflated 87%)
  adding: classified2/com/architecture-design-services.txt (deflated 87%)
  adding: classified2/com/artistic-design-services.txt (deflated 87%)
  adding: classified2/com/asset-property-inspection-verification.txt (deflated 88%)
  adding: classified2/com/biomedi

#Not needed

In [ ]:
#installing pypy3
!sudo add-apt-repository ppa:pypy/ppa
!sudo apt update
!sudo apt install pypy3
!pypy3 get-pip.py


 The latest release of PyPy packaged for stable Ubuntu releases.

This PPA also contains builds of CFFI for cPython - python-cffi.
 More info: https://launchpad.net/~pypy/+archive/ubuntu/ppa
Press [ENTER] to continue or Ctrl-c to cancel adding it.

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [62.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.u

In [ ]:
# !pypy3 -m pip install requests
# !pypy3 -m pip install beautifulsoup4
!pypy3 -m pip install gensim
#!pypy3 -m pip install pandas


distutils: /usr/lib/pypy3/../../local/lib/pypy3.7/dist-packages
sysconfig: /usr/lib/pypy3/site-packages
distutils: /usr/lib/pypy3/../../local/lib/pypy3.7/dist-packages
sysconfig: /usr/lib/pypy3/site-packages
distutils: /usr/lib/pypy3/../../local/include
sysconfig: /usr/lib/pypy3/include/UNKNOWN
distutils: /usr/lib/pypy3/../../local/bin
sysconfig: /usr/lib/pypy3/bin
distutils: /usr/lib/pypy3/../../local
sysconfig: /usr/lib/pypy3
user = False
home = None
root = None
prefix = None
  Using cached gensim-4.0.1.tar.gz (23.1 MB)
  Using cached numpy-1.21.1-pp37-pypy37_pp73-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.2 MB)
  Using cached scipy-1.7.1.tar.gz (36.1 MB)
  distutils: /tmp/pip-build-env-9lw_wv0_/normal/site-packages
  sysconfig: /tmp/local/lib/pypy3.7/lib-python
  distutils: /tmp/pip-build-env-9lw_wv0_/normal/site-packages
  sysconfig: /tmp/local/lib/pypy3.7/lib-python
  user = False
  home = None
  root = None
  prefix = '/tmp/pip-build-env-9lw_wv0_/normal'
  distutils: /tmp

In [ ]:
#we load the classification excel file with pandas
#We load the alphabet sheet which has the data we need to populate the game and automatically analyze sounds and shapes of letters
import pandas as pd

xls = pd.ExcelFile('290721.xlsx')
categories_sheet=pd.read_excel(xls, 'categories',keep_default_na=False)
parents_sheet=pd.read_excel(xls, 'parents',keep_default_na=False)

print(categories_sheet.keys())
print(xls.sheet_names)

keyword_dict={}
for row in categories_sheet.iterrows():
  cur_obj={}
  row_dict=row[1].to_dict()
  cat_name=row_dict["Category"]
  if cat_name=="": continue
  keywords=row_dict["Keywords"]
  parent1=row_dict["Parent1"]
  alias1=row_dict["Alias1"]
  parent2=row_dict["Parent2"]
  alias2=row_dict["Alias2"]
  parent3=row_dict["Parent3"]
  alias3=row_dict["Alias3"]
  print(cat_name, "=", keywords)

  # key=row_dict["arabic_script_extended"]
  # letter_ar=row_dict["arabic_script"]


Index(['#', 'Category', 'Parent1', 'Alias1', 'Parent2', 'Alias2', 'Parent3',
       'Alias3', 'Keywords', 'Accuracy of First Page (out of 20)',
       'Accuracy of First Page (%)', 'Action (Select)', 'Action Complete?',
       'Location Info Important to Buyers?',
       'Action after next run (if applicable)', 'Remarks', 'Unnamed: 16',
       'Unnamed: 17'],
      dtype='object')
['parents', 'categories', 'Parked - July 2021']
Accounting Services = Accounting, Bookkeeping, Tax, Records, Returns, Payroll, Lodgement, Statement
Buses & Vans = Bus, Van, Coach, Minibus, Mini, Bus, Sales, Rental, Leasing, Hire, maintenance
Trucks & Utes = Truck, Ute, Sales, Rental, Leasing, Hire, maintenance
Heavy Vehicle Maintenance = Truck, Bus, Trailer, Repairs, Maintenance, Diesel, Heavy, Fleet
Earth Moving Equipment = Earthmoving, Bobcat, Excavator, Loader, Trencher, Dumper, Sales, Rental, Leasing, Hire, maintenance, Backhoe
Financial Audit & Compliance Services = Audit, Compliance, Financial, Money, R

In [ ]:
#4- And now for the main step - caching the content from the list of URLs
#https://www.reddit.com/r/learnpython/comments/8b4q6f/scraping_multiple_web_pages_simultaneously/
#now we adapt it
import concurrent.futures
import urllib.request
from utils.extraction_utils import *

def gen_url_list(fname,start_i=0,n_pages=50,exclude_dict={}):
  fopen=open(fname)
  for li,line in enumerate(fopen):
    if line=="": 
      fopen.close()
      return
    #if li>=20: break
    
    if li<start_i: continue
    if li>=start_i+n_pages: break
    if li%10000==0: print("file is now at:", li)
    cur_url=line.strip("\n\r")
    if exclude_dict.get(cur_url)==True: continue
    yield cur_url
  fopen.close()

crawl_dir="crawl"
run_dir="aug21" #aug21_au
main_dir_path=os.path.join(crawl_dir,run_dir)

url_list_fname="all_urls.txt"
cached_content_fname="cached_all.txt"

cached_url_dict={}
url_list_fpath=os.path.join(main_dir_path,url_list_fname) #"all_au_urls.txt"
cached_content_fpath=os.path.join(main_dir_path,cached_content_fname) #"cached_all_au.txt"

if os.path.exists(cached_content_fpath):
  cached_fopen=open(cached_content_fpath)
  for i,f in enumerate(cached_fopen):
    # f_split=f.split("<br>")
    url=f.split("<br>")[0]
    if not url.startswith("http"): continue
    cached_url_dict[url]=True
  cached_fopen.close()

n_urls=file_len(url_list_fpath)
print("number of urls to process:",n_urls)

batch_size=1000
for i_ in range(0,n_urls,batch_size): #the end number should be the size of the file being read
  print("current i",i_)
  URLS=gen_url_list(url_list_fpath,i_,batch_size,cached_url_dict)

  # We can use a with statement to ensure threads are cleaned up promptly
  content_list=[]
  with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
      # Start the load operations and mark each future with its URL
      #future_to_url = {executor.submit(load_url, url, 60): url for url in URLS}
      future_to_url = {executor.submit(get_content, url): url for url in URLS if cached_url_dict.get(url)!=True}
      for future in concurrent.futures.as_completed(future_to_url):
          url = future_to_url[future]
          try:
              data = future.result()
          except Exception as exc:
              #print('%r generated an exception: %s' % (url, exc))
              pass
          else:
              #print('%r page is %d bytes' % (url, len(data)))
              content_list.append(data)
              #if len(content_list)%500==0: print("current content list length", len(content_list))
              #print(len(content_list))
  cached_content_fopen=open(cached_content_fpath,"a")
  
  for cl in content_list:
    cached_content_fopen.write(cl+"\n")
  cached_content_fopen.close()

/content/drive/My Drive/KM/utils/extraction_utils.py:49: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 49 of the file /content/drive/My Drive/KM/utils/extraction_utils.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup=BeautifulSoup(self.html_content)


number of urls to process: 553586
current i 0
file is now at: 0
current i 1000
current i 2000
current i 3000
current i 4000
current i 5000
current i 6000
current i 7000
current i 8000
current i 9000
current i 10000
file is now at: 10000
current i 11000
current i 12000
current i 13000
current i 14000
current i 15000
current i 16000
current i 17000
current i 18000
current i 19000
current i 20000
file is now at: 20000
current i 21000
current i 22000
current i 23000
current i 24000
current i 25000
current i 26000
current i 27000
current i 28000
current i 29000
current i 30000
file is now at: 30000
current i 31000
current i 32000
current i 33000
current i 34000
current i 35000
current i 36000
current i 37000
current i 38000
current i 39000
current i 40000
file is now at: 40000
current i 41000
current i 42000
current i 43000
current i 44000
current i 45000
current i 46000
current i 47000
current i 48000
current i 49000
current i 50000
file is now at: 50000
current i 51000
current i 52000
cur

In [ ]:
#Now run the caching script on the small data with 60 workers and a batch of 10000
!chmod +x utils/caching_utils.py
!utils/caching_utils.py #with 10000 batch - 60 workers
#completed in 433.3331551551819 seconds

number of urls to process: 553586
completed in 10.773498058319092 seconds


In [ ]:
!chmod +x utils/caching_utils.py
!utils/caching_utils.py #with 1000 batch - 30 workers
#completed in 588.3326060771942 seconds

number of urls to process: 7519
current i 0 time: 0.0
current i 1000 time: 68.52
current i 2000 time: 87.21
current i 3000 time: 78.14
current i 4000 time: 72.93
current i 5000 time: 78.81
current i 6000 time: 77.6
current i 7000 time: 73.02
completed in 588.3326060771942 seconds


In [ ]:
#with 1000 batch - 50 workers
!utils/caching_utils.py
#completed in 456.4794147014618 seconds

number of urls to process: 7519
current i 0 time: 0.0
current i 1000 time: 50.1
current i 2000 time: 71.2
current i 3000 time: 56.57
current i 4000 time: 57.38
current i 5000 time: 57.3
current i 6000 time: 57.81
current i 7000 time: 54.67
completed in 456.4794147014618 seconds


In [ ]:
#with 1000 batch - 100 workers
!utils/caching_utils.py
#completed in 463.1788227558136 seconds

number of urls to process: 7519
current i 0 time: 0.0
current i 1000 time: 52.63
current i 2000 time: 69.95
current i 3000 time: 57.14
current i 4000 time: 56.31
current i 5000 time: 54.81
current i 6000 time: 57.28
current i 7000 time: 70.12
completed in 463.1788227558136 seconds


In [ ]:
#with 1000 batch - 40 workers
!utils/caching_utils.py
#completed in 507.22553849220276 seconds

number of urls to process: 7519
current i 0 time: 0.0
current i 1000 time: 57.18
current i 2000 time: 72.82
current i 3000 time: 63.35
current i 4000 time: 75.02
current i 5000 time: 63.45
current i 6000 time: 64.5
current i 7000 time: 58.95
completed in 507.22553849220276 seconds


In [ ]:
#with 1000 batch - 60 workers
!utils/caching_utils.py
#completed in 448.39253640174866 seconds

number of urls to process: 7519
current i 0 time: 0.0
current i 1000 time: 49.13
current i 2000 time: 64.02
current i 3000 time: 55.23
current i 4000 time: 66.33
current i 5000 time: 54.71
current i 6000 time: 55.32
current i 7000 time: 50.48
completed in 448.39253640174866 seconds


In [ ]:
!pypy3 utils/caching_utils.py

number of urls to process: 7519
current i 0 time: 0.0
/content/drive/MyDrive/KM/utils/extraction_utils.py:49: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 49 of the file /content/drive/MyDrive/KM/utils/extraction_utils.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  soup=BeautifulSoup(self.html_content)
current i 1000 time: 69.56
current i 2000 time: 81.82
current i 3000 time: 76.69
current i 4000 time: 76.78
current i 5000 time: 80.28
current i 6000 time: 77.0
current i 7000 time: 76.4
completed in 580.5658779144287 seconds


In [ ]:
# !tail -5 crawl/aug21/cached_all.txt
# !wc crawl/aug21/cached_all.txt
fpath="crawl/aug21/cached_all.txt"
url_dict={}
fopen=open(fpath)
for i,f in enumerate(fopen):
  # f_split=f.split("<br>")
  url=f.split("<br>")[0]
  if not url.startswith("http"): continue
  url_dict[url]=True
fopen.close()
print(len(url_dict.keys()) )

325761


In [ ]:
# !pypy3 -m pip install requests
# !pypy3 -m pip install beautifulsoup4
# !pypy3 -m pip install gensim
!pypy3 -m pip install futures

distutils: /usr/lib/pypy3/../../local/lib/pypy3.7/dist-packages
sysconfig: /usr/lib/pypy3/site-packages
distutils: /usr/lib/pypy3/../../local/lib/pypy3.7/dist-packages
sysconfig: /usr/lib/pypy3/site-packages
distutils: /usr/lib/pypy3/../../local/include
sysconfig: /usr/lib/pypy3/include/UNKNOWN
distutils: /usr/lib/pypy3/../../local/bin
sysconfig: /usr/lib/pypy3/bin
distutils: /usr/lib/pypy3/../../local
sysconfig: /usr/lib/pypy3
user = False
home = None
root = None
prefix = None
  distutils: /usr/lib/pypy3/../../local/include
  sysconfig: /usr/lib/pypy3/include/futures


In [ ]:
fname="crawl/aug21/all_urls.txt"
fopen=open(fname)
for i,f in enumerate(fopen):
  if i>100: break
  print([f])
  # url=f.split("<br>")[0]
  # if not url.startswith("http"):  print([f])
fopen  
fopen.close()


['https://embryology.med.unsw.edu.au\n']
['https://emed.med.unsw.edu.au\n']
['https://ethics.med.unsw.edu.au\n']
['https://iser.med.unsw.edu.au\n']
['https://medicalsciences.med.unsw.edu.au\n']
['https://medprogram.med.unsw.edu.au\n']
['https://ndarc.med.unsw.edu.au\n']
['http://php.med.unsw.edu.au\n']
['https://powcs.med.unsw.edu.au\n']
['https://rmlab.med.unsw.edu.au\n']
['https://sims.med.unsw.edu.au\n']
['https://sph.med.unsw.edu.au\n']
['https://sphcm.med.unsw.edu.au\n']
['https://staff.med.unsw.edu.au\n']
['https://stgcs.med.unsw.edu.au\n']
['https://stvcs.med.unsw.edu.au\n']
['https://swscs.med.unsw.edu.au\n']
['https://teachingtools.med.unsw.edu.au\n']
['https://wch.med.unsw.edu.au\n']
['http://web.med.unsw.edu.au\n']
['https://membrane.unsw.edu.au\n']
['https://mentoring.unsw.edu.au\n']
['https://meridian180.unsw.edu.au\n']
['http://merlin.unsw.edu.au\n']
['https://mhdcd.unsw.edu.au\n']
['http://mining.unsw.edu.au\n']
['http://mm2015.unsw.edu.au\n']
['https://music.unsw.edu.au

In [ ]:
!wc -l crawl/aug21/gov.au.txt
!wc -l crawl/aug21/cached_gov11.txt

7519 crawl/aug21/gov.au.txt
7519 crawl/aug21/cached_gov11.txt


#Dump

In [ ]:

#https://index.commoncrawl.org/CC-MAIN-2021-10/
#/CC-MAIN-2021-25 - June 2021
#!chmod +x utils/cdx-index-client.py


#!./cdx-index-client.py -c CC-MAIN-2021-10 *.io --show-num-pages
#!./cdx-index-client.py -c CC-MAIN-2021-10 *.gov.au --show-num-pages
#!./cdx-index-client.py -c CC-MAIN-2021-10 *.edu.au --fl url -z -d edu.au
#!./cdx-index-client.py -c CC-MAIN-2021-10 *.org.au --fl url -z -d org.au
# crawl_version="CC-MAIN-2021-25"
# crawl_version="CC-MAIN-2021-31"  #July 21
# crawl_dir="crawl"
# run_dir="oct21_au" #aug21_au
# urls_dir="urls" #<<<<<< add a subdirectry to contain only the urls

# command = "ntpq -p"

#domain="edu.au"
#for domain in ["edu.au","com.au","net.au","gov.au","org.au"]:
  # os.system("utils/cdx-index-client.py -c %s *.%s --show-num-pages"%(crawl_version,domain))
  # #out = check_output(["ntpq", "-p"])

  # #full_dir_path=os.path.join(crawl_dir,run_dir,domain)

  # os.system("utils/cdx-index-client.py -c %s *.%s --fl url -z -d %s"%(crawl_version,domain,full_dir_path))
#!./cdx-index-client.py -c CC-MAIN-2021-10 *.com.au --fl url -z -d com.au

#OLD separate steps

In [ ]:
#2- Then extract the list of urls from the results of the query (stored in gzip files) into text files 
import gzip, os, sys
#!ls
sys.path.append("utils")
from utils.extraction_utils import *

crawl_dir="crawl"
#run_dir="aug21" #aug21_au
run_dir="oct21_au" #aug21_au
urls_dir="urls" #<<<<<< add a subdirectry to contain only the urls

main_dir_path=os.path.join(crawl_dir,run_dir)
counter_dict={}

for cur_dir in os.listdir(main_dir_path):
  #full_dpath=os.path.join(main_dir_path,cur_dir)
  full_dpath=os.path.join(main_dir_path,cur_dir,urls_dir)
  if not os.path.isdir(full_dpath): continue
  url_counter=0
  print(full_dpath)
  out_txt_fname=cur_dir+".txt"
  out_txt_fpath=os.path.join(main_dir_path,out_txt_fname)
  print("output file:", out_txt_fpath)
  list_fopen=open(out_txt_fpath,"w")
  files=os.listdir(full_dpath)
  print(len(files))
  for i,fname in enumerate(files):
    used_urls=[]
    if i%100==0: print(cur_dir, i, "out of", len(files))
    #print(">>>>>>> fname",fname)
    fpath=os.path.join(full_dpath,fname)
    tmp_fopen=open(fpath)
    for tf in tmp_fopen:
      #print([tf])
      #print(tf.split())
      
      cur_url=tf.split()[-1].strip('"{}')
      # print(cur_url)
      if cur_url.split(".")[-1].lower() in ["pdf","png","js","css","jpg"]: continue
      if cur_url.endswith("robots.txt") or cur_url.endswith("wp-admin"): continue
      if "ezproxy" in cur_url: continue
        # print(">>>>>>> URL:", cur_url)
    # gz_open=gzip.open(fpath,'r')
    
    # for gz in gz_open:
    #   #print(str(gz))
    #   cur_url=str(gz)[2:-3]
      bare_url=get_bare_url(cur_url)
      # print(bare_url)
      if bare_url in used_urls: continue
      used_urls.append(bare_url)

      #print(bare_url,len(bare_url.split(".")))
      
      if "com" in cur_dir and len(bare_url.split("."))>3: continue #skip subdomains for .com websites
      if "net" in cur_dir and len(bare_url.split("."))>3: continue #skip subdomains for .net websites
      new_url=cur_url.split("//")[0]+"//"+bare_url  

      url_counter+=1
      #print(">>>>", new_url)
      list_fopen.write(new_url+"\n")
  list_fopen.close()
  counter_dict[cur_dir]=url_counter

for key,val in counter_dict.items():
  print(key, val)





crawl/oct21_au/edu.au/urls
output file: crawl/oct21_au/edu.au.txt
147
edu.au 0 out of 147
edu.au 100 out of 147
crawl/oct21_au/com.au/urls
output file: crawl/oct21_au/com.au.txt
2296
com.au 0 out of 2296
com.au 100 out of 2296
com.au 200 out of 2296
com.au 300 out of 2296
com.au 400 out of 2296
com.au 500 out of 2296
com.au 600 out of 2296
com.au 700 out of 2296
com.au 800 out of 2296
com.au 900 out of 2296
com.au 1000 out of 2296
com.au 1100 out of 2296
com.au 1200 out of 2296
com.au 1300 out of 2296
com.au 1400 out of 2296
com.au 1500 out of 2296
com.au 1600 out of 2296
com.au 1700 out of 2296
com.au 1800 out of 2296
com.au 1900 out of 2296
com.au 2000 out of 2296
com.au 2100 out of 2296
com.au 2200 out of 2296
crawl/oct21_au/net.au/urls
output file: crawl/oct21_au/net.au.txt
60
net.au 0 out of 60
crawl/oct21_au/gov.au/urls
output file: crawl/oct21_au/gov.au.txt
139
gov.au 0 out of 139
gov.au 100 out of 139
crawl/oct21_au/org.au/urls
output file: crawl/oct21_au/org.au.txt
147
org.au 

In [ ]:
#Now combining all the text files with url lists into one big list
import shutil
crawl_dir="crawl"
run_dir="aug21" #aug21_au
run_dir="oct21_au"
main_dir_path=os.path.join(crawl_dir,run_dir)
txt_files=[os.path.join(main_dir_path,f0) for f0 in os.listdir(main_dir_path) if f0.endswith(".txt")]
out_fpath=os.path.join(main_dir_path,"all_urls.txt")
print(txt_files)
print(out_fpath)
with open(out_fpath,'wb') as wfd:
    for f in txt_files:
        with open(f,'rb') as fd:
            shutil.copyfileobj(fd, wfd)

# !cat "edu.au.txt" "com.au.txt" "gov.au.txt" "org.au.txt" > "all_au_urls.txt"

['crawl/oct21_au/edu.au.txt', 'crawl/oct21_au/com.au.txt', 'crawl/oct21_au/net.au.txt', 'crawl/oct21_au/gov.au.txt', 'crawl/oct21_au/org.au.txt']
crawl/oct21_au/all_urls.txt


In [ ]:
import h5py, os, json
import numpy as np
f = h5py.File("mytestfile.hdf5", "w")
dset = f.create_dataset("mydataset", (100,), dtype='i')
grp = f.create_group("subgroup")
dset2 = grp.create_dataset("another_dataset", (50,), dtype='f')
print(dset2.name)
dset3 = f.create_dataset('com/agriculture/ai', (1,), dtype='i')
dset4 = f.create_dataset('com/agriculture/manual', (1,), dtype='i')
dset5 = f.create_dataset('com/construction', (1,), dtype='i')
domains=["com","gov","edu","org"]
categories=["agriculture","construction","manufaturing"]
sources=["ai","manual"]
for cat in categories:
  for dom in domains:
    for src in sources:
      cur_path=os.path.join(cat,dom,src)
      dset_new=f.create_dataset(cur_path, (1,), dtype='b')
      attrs_dict={"cat":cat,"dom":dom}
      dset_new.attrs["info"]=json.dumps(attrs_dict)

f.close()

/subgroup/another_dataset


In [ ]:
def printname(name):
  print(name)
f = h5py.File("mytestfile.hdf5", "r+")
#dataset_three = f['agriculture/com/ai']
dataset_three = f['agriculture/com']
cur_info={"size":57}
dataset_three.attrs["info"]=json.dumps(cur_info)
print(dataset_three)
print(dataset_three.attrs["info"])
# for it0 in f.items():
#   print(it0)
#print(dataset_three)

#f.visit(printname)
f.close()

<HDF5 group "/agriculture/com" (2 members)>
{"size": 57}


In [ ]:
 f.close()

In [ ]:
import h5py, os, json
import numpy as np
root_dir="crawl/aug21/classified4"
domains=["com","gov","edu","org"]

hf = h5py.File("classified-aug21.hdf5", "w")
for dom in domains:
  cur_dir=os.path.join(root_dir,dom)
  cur_files=os.listdir(cur_dir)
  print(dom, len(cur_files))
  for f_counter,fname in enumerate(cur_files):
    if f_counter%20==0: print(f_counter)
    cur_fpath=os.path.join(cur_dir,fname)
    cur_cat=fname.split(".")[0]
    print(dom, cur_cat, cur_fpath)
    cat_fopen=open(cur_fpath)
    for i,line in enumerate(cat_fopen):
      #if i>4: break
      line_split=line.strip().split("\t")
      if len(line_split)!=3: continue
      cur_url,cur_score_str,cur_pred_json=line_split
      cur_obj=json.loads(cur_pred_json)
      cur_obj["url"]=cur_url
      cur_obj["score"]=float(cur_score_str)
      item_id=str(i).zfill(6)
      h5_path=os.path.join(dom,cur_cat,"ai",item_id)
      #print(h5_path)
      dset_new=hf.create_dataset(h5_path, (1,), dtype='b')
      dset_new.attrs["info"]=json.dumps(cur_obj)
      
      #print(cur_obj)
      #print(line_split)
    cat_fopen.close()



hf.close()

com 107
0
com 3d-printing crawl/aug21/classified4/com/3d-printing.txt
com accounting-services crawl/aug21/classified4/com/accounting-services.txt
com advertising-services crawl/aug21/classified4/com/advertising-services.txt
com animal-nutrition-health-products crawl/aug21/classified4/com/animal-nutrition-health-products.txt
com animal-transport-logistics crawl/aug21/classified4/com/animal-transport-logistics.txt
com animal-veterinary-services crawl/aug21/classified4/com/animal-veterinary-services.txt
com architecture-design-services crawl/aug21/classified4/com/architecture-design-services.txt
com artistic-design-services crawl/aug21/classified4/com/artistic-design-services.txt
com asset-property-inspection-verification crawl/aug21/classified4/com/asset-property-inspection-verification.txt
com biomedical-pharmaceutical-research crawl/aug21/classified4/com/biomedical-pharmaceutical-research.txt
com building-property-insurance crawl/aug21/classified4/com/building-property-insurance.txt
co

In [ ]:
hf.close()

In [ ]:
import time
t0=time.time()
hf = h5py.File("classified-aug21.hdf5", "r+")
t1=time.time()
print("opened h5 file", t1-t0)

t0=time.time()
test_dataset = hf['com/accounting-services/ai']
t1=time.time()
print("opened current dataset", t1-t0)

t0=time.time()
for td in test_dataset:
  print(td, test_dataset[td].attrs["info"])
#print(test_dataset)
t1=time.time()
print(len(test_dataset))
print("opened dataset items", t1-t0)

hf.close()

opened h5 file 0.0030164718627929688
opened current dataset 0.0017628669738769531
000000 {"preds": [["accounting-services", 0.8437], ["financial-audit-compliance-services", 0.6438], ["lending-services", 0.6212], ["investment-finance-advisory-services", 0.5671], ["investment-fund-management-services", 0.5661]], "top_words": ["tax", "returns", "bookkeeping", "elsie", "cogar", "quickbooks", "general", "company", "business", "activity"], "cat_words": ["bookkeeping", "accounting", "tax", "payments", "returns"], "url": "http://remoteaccounting.com.au", "score": 0.8437}
000001 {"preds": [["accounting-services", 0.8307], ["financial-audit-compliance-services", 0.7243], ["investment-finance-advisory-services", 0.7014], ["lending-services", 0.6964], ["business-insurance", 0.643]], "top_words": ["tax", "cabramatta", "accounting", "bookkeeping", "business", "accountants", "agent", "taxation", "payroll", "income"], "cat_words": ["bookkeeping", "payroll", "accounting", "tax", "accountants"], "url": 

In [ ]:
hf.close()

In [ ]:
import time

fpath="crawl/aug21/classified4/com/it-hardware-retail.txt"
#fpath="crawl/aug21/classified4/org/web-development-services.txt"

out_fpath="crawl/aug21/test.txt"
out_fopen=open(out_fpath,"w")
t0=time.time()
counter=0
max_len=0
fopen=open(fpath)
for i,f in enumerate(fopen):
  counter+=1
  url=f.split("\t")[0]
  if  len(url)>max_len: max_len=len(url)
  padded_url_line=url.ljust(99)+"\n"
  out_fopen.write(padded_url_line)

fopen.close()
out_fopen.close()
t1=time.time()
elapsed=t1-t0
print("lines:", counter,"max_len:",max_len,"time:",elapsed)

lines: 87673 max_len: 57 time: 0.2629213333129883


In [ ]:
out_fpath="crawl/aug21/test.txt"
line_i=8900
file_loc=line_i*100
out_fopen=open(out_fpath)
out_fopen.seek(file_loc)
line=out_fopen.readline()
print(line.strip(), len(line))
out_fopen.close()

out_fopen=open(out_fpath)
for i,f in enumerate(out_fopen):
  if i==line_i:
    print(f.strip(), len(f))
    break
out_fopen.close()

https://v-daq.com.au 100
https://v-daq.com.au 100


In [ ]:
import tempfile, shutil, os, time
out_fpath="crawl/aug21/test.txt"
out_fpath2="crawl/aug21/test2.txt"
out_fpath3="crawl/aug21/test3.txt"
test_line="https://brainsolutions.com.au"
#test_line="https://fittrack.com.au"
test_line="https://infratherm.com.au"
tmp = tempfile.NamedTemporaryFile(delete=True)
shutil.copy2(out_fpath, tmp.name)
#shutil.copy2(out_fpath, out_fpath2)
t0=time.time()
write_fopen=open(out_fpath,"w")
tmp_fpath=tmp.name
fopen=open(tmp_fpath)
for line in fopen:
  if line.startswith(test_line): pass
  else: write_fopen.write(line)
fopen.close()
write_fopen.close()
t1=time.time()
print(tmp.name, t1-t0)

/tmp/tmp2f3phbgh 0.12214231491088867


In [ ]:
!wc "crawl/aug21/test.txt"
!wc "crawl/aug21/test2.txt"

  87670   87670 8767000 crawl/aug21/test.txt
  87673   87673 8767300 crawl/aug21/test2.txt


In [ ]:
import tempfile, shutil, os, time
def remove_line(str0,fpath0): #remove a line that starts with certin str
  tmp = tempfile.NamedTemporaryFile(delete=True)
  shutil.copy2(fpath0, tmp.name)
  write_fopen=open(fpath0,"w")
  tmp_fpath=tmp.name
  fopen=open(tmp_fpath)
  for li in fopen:
    if li.startswith(str0): pass
    else: write_fopen.write(li)
  fopen.close()
  write_fopen.close()

def get_line(line_i0,fpath0,line_size=100): #from a file with fixed line size
  fopen0=open(fpath0)
  fopen0.seek(line_i0*line_size)
  cur_line=fopen0.readline().strip()
  fopen0.close()
  return cur_line

def get_n_lines(fpath0,line_size=100): #get number of lines of a file with fixed line size
  file_size=os.path.getsize(fpath0)
  return int(file_size/line_size)

def insert_sorted(str0,fpath0,line_size=100): #insert string in a file to maintain alphabetical order
  cur_line=str0.ljust(line_size-1)+"\n"
  output=False
  if not os.path.exists(fpath0):
    fopen=open(fpath0,"w")
    fopen.write(cur_line)
    fopen.close()
    return output
  #print("now adding to an existing file")
  tmp = tempfile.NamedTemporaryFile(delete=True)
  shutil.copy2(fpath0, tmp.name)
  write_fopen=open(fpath0,"w")
  tmp_fpath=tmp.name
  fopen=open(tmp_fpath)
  added=False
  for li in fopen:
    if li.strip()==str0.strip():
      output=False
    elif str0<li: #if the string to be inserted is smaller than the current line (before it in the sorting order)
      if added==False:
        write_fopen.write(cur_line) #write the string first, then the current line
        write_fopen.write(li)
        added=True
      else: write_fopen.write(li) #if it was already added

    else:
      write_fopen.write(li)
  if added==False: write_fopen.write(cur_line) #if the string is not added yet, then add it at the end of the line
  fopen.close()
  write_fopen.close()
  return output


cur_i=980
cur_line=get_line(cur_i,cur_fpath)
print(cur_line)

#cur_line="https://v-daq.com.au"
cur_fpath="crawl/aug21/test.txt"
t0=time.time()
remove_line(cur_line,cur_fpath)
t1=time.time()
print(t1-t0)
!wc "crawl/aug21/test.txt"
print(get_n_lines("crawl/aug21/test.txt"))

https://siliconchip.com.au
0.10106754302978516
  87659   87659 8765900 crawl/aug21/test.txt
87659


In [ ]:
out_fpath2="crawl/aug21/test2.txt"
out_fpath3="crawl/aug21/test3.txt"
shutil.copy2(out_fpath2, out_fpath3)
t0=time.time()
remove_line("https://v",out_fpath3)
t1=time.time()
print(t1-t0)
!wc "crawl/aug21/test3.txt"

0.11022090911865234
  86559   86559 8655900 crawl/aug21/test3.txt


In [ ]:
print(os.path.getsize("crawl/aug21/test.txt"))
!wc "crawl/aug21/test.txt"

8766700
  87667   87667 8766700 crawl/aug21/test.txt


In [ ]:


test_fpath="crawl/aug21/tes0.txt"
t0=time.time()
insert_sorted("yellow",test_fpath)
t1=time.time()
print(t1-t0)
fopen=open(test_fpath)
print(fopen.read())
fopen.close()

0.007874727249145508
#@#                                                                                                
111                                                                                                
555                                                                                                
NO                                                                                                 
SERIOUSLY                                                                                          
ab0                                                                                                
abba                                                                                               
about time                                                                                         
celebrate                                                                                          
clear                                                                          

In [ ]:
def reverse_url(full_url):
  bare_url=full_url.replace("http://","").replace("https://","")
  if bare_url.startswith("www."): bare_url=bare_url.replace("www.","")
  if bare_url.startswith("www1."): bare_url=bare_url.replace("www1.","")
  if bare_url.startswith("www2."): bare_url=bare_url.replace("www2.","")  
  url_split=bare_url.split("/")
  first_part=url_split[0]
  second_part="/".join(url_split[1:])
  first_part=".".join(reversed(first_part.split(".")))
  if second_part: return first_part+"/"+second_part
  else: return first_part

url='https://search.google.com/search-console?resource_id=https%3A%2F%2Fwww.champolu.com%2Fcontent%2F'
url="abc.gov.au/what/is/else"
reverse_url(url)

'au.gov.abc/what/is/else'

In [ ]:
!wget https://johnvansickle.com/ffmpeg/builds/ffmpeg-git-amd64-static.tar.xz

--2021-10-23 14:25:27--  https://johnvansickle.com/ffmpeg/builds/ffmpeg-git-amd64-static.tar.xz
Resolving johnvansickle.com (johnvansickle.com)... 107.180.57.212
Connecting to johnvansickle.com (johnvansickle.com)|107.180.57.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40835464 (39M) [application/x-xz]
Saving to: ‘ffmpeg-git-amd64-static.tar.xz’

ffmpeg-git-amd64-st 100%[===================>]  38.94M  40.2MB/s    in 1.0s    

2021-10-23 14:25:28 (40.2 MB/s) - ‘ffmpeg-git-amd64-static.tar.xz’ saved [40835464/40835464]



In [ ]:
!wget https://johnvansickle.com/ffmpeg/builds/ffmpeg-git-amd64-static.tar.xz.md5

--2021-10-23 14:25:49--  https://johnvansickle.com/ffmpeg/builds/ffmpeg-git-amd64-static.tar.xz.md5
Resolving johnvansickle.com (johnvansickle.com)... 107.180.57.212
Connecting to johnvansickle.com (johnvansickle.com)|107.180.57.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65 [application/x-xz]
Saving to: ‘ffmpeg-git-amd64-static.tar.xz.md5’

ffmpeg-git-amd64-st 100%[===================>]      65  --.-KB/s    in 0s      

2021-10-23 14:25:49 (5.65 MB/s) - ‘ffmpeg-git-amd64-static.tar.xz.md5’ saved [65/65]



In [ ]:
!md5sum -c ffmpeg-git-amd64-static.tar.xz.md5

ffmpeg-git-amd64-static.tar.xz: OK


In [ ]:
!tar xvf ffmpeg-git-amd64-static.tar.xz

ffmpeg-git-20211013-amd64-static/
ffmpeg-git-20211013-amd64-static/GPLv3.txt
ffmpeg-git-20211013-amd64-static/manpages/
ffmpeg-git-20211013-amd64-static/manpages/ffmpeg-all.txt
ffmpeg-git-20211013-amd64-static/manpages/ffmpeg-scaler.txt
ffmpeg-git-20211013-amd64-static/manpages/ffmpeg-resampler.txt
ffmpeg-git-20211013-amd64-static/manpages/ffmpeg-filters.txt
ffmpeg-git-20211013-amd64-static/manpages/ffprobe.txt
ffmpeg-git-20211013-amd64-static/manpages/ffmpeg-devices.txt
ffmpeg-git-20211013-amd64-static/manpages/ffmpeg-utils.txt
ffmpeg-git-20211013-amd64-static/manpages/ffmpeg-protocols.txt
ffmpeg-git-20211013-amd64-static/manpages/ffmpeg-codecs.txt
ffmpeg-git-20211013-amd64-static/manpages/ffmpeg-bitstream-filters.txt
ffmpeg-git-20211013-amd64-static/manpages/ffmpeg.txt
ffmpeg-git-20211013-amd64-static/manpages/ffmpeg-formats.txt
ffmpeg-git-20211013-amd64-static/ffprobe
ffmpeg-git-20211013-amd64-static/qt-faststart
ffmpeg-git-20211013-amd64-static/model/
ffmpeg-git-20211013-amd64-stat

In [ ]:
!ls

 14october21.xlsx
'2020 09 07 Classification R2.gsheet'
 2021-10-16.xlsx
 26april.zip
 290721-old.xlsx
 290721.xlsx
 all_au_cache2.txt
 all_au_urls.txt
 au_cached
 au_cache.txt
 au_pages.shelve.db
 au_urls.txt
 au-websites-gov-edu-org.gsheet
 b2b_classification_platform.ipynb
 b2web.ipynb
 cached_all_au.txt
'Capturing Knowledge Challenge - Final.gdoc'
 cdx-index-client.py
 classification.gsheet
 classified5.txt
 classified6.gsheet
 classified6.tsv
 classified6.txt
 classified7.txt
 classified8.txt
 classified9.txt
 classified-aug21.hdf5
 classified_list4website2.tsv
 classified_list4website_full0.tsv
 classified_list4website_only_top.tsv
 classified_list4website.tsv
 crawl
 data_dict1.json
 data_dict.json
 data_dict_new123.json
 data_dict_new2.json
 data_dict_new.json
 data_dict_new_old.json
 data.hdf5
 edu.au_au_cache.txt
'Email Heap Executive Summary.gdoc'
'Email Heap Slide Deck.gslides'
 ffmpeg-git-20211013-amd64-static
 ffmpeg-git-amd64-static.tar.xz
 ffmpeg-git-amd64-static.tar.xz

In [ ]:
!zip -r ffmpeg.zip ffmpeg-git-20211013-amd64-static

  adding: ffmpeg-git-20211013-amd64-static/ (stored 0%)
  adding: ffmpeg-git-20211013-amd64-static/GPLv3.txt (deflated 66%)
  adding: ffmpeg-git-20211013-amd64-static/manpages/ (stored 0%)
  adding: ffmpeg-git-20211013-amd64-static/manpages/ffmpeg-all.txt (deflated 73%)
  adding: ffmpeg-git-20211013-amd64-static/manpages/ffmpeg-scaler.txt (deflated 64%)
  adding: ffmpeg-git-20211013-amd64-static/manpages/ffmpeg-resampler.txt (deflated 66%)
  adding: ffmpeg-git-20211013-amd64-static/manpages/ffmpeg-filters.txt (deflated 73%)
  adding: ffmpeg-git-20211013-amd64-static/manpages/ffprobe.txt (deflated 68%)
  adding: ffmpeg-git-20211013-amd64-static/manpages/ffmpeg-devices.txt (deflated 71%)
  adding: ffmpeg-git-20211013-amd64-static/manpages/ffmpeg-utils.txt (deflated 67%)
  adding: ffmpeg-git-20211013-amd64-static/manpages/ffmpeg-protocols.txt (deflated 68%)
  adding: ffmpeg-git-20211013-amd64-static/manpages/ffmpeg-codecs.txt (deflated 72%)
  adding: ffmpeg-git-20211013-amd64-static/manpa

In [ ]:
!wget https://www.johnvansickle.com/ffmpeg/old-releases/ffmpeg-4.1.4-amd64-static.tar.xz

--2021-10-23 14:41:04--  https://www.johnvansickle.com/ffmpeg/old-releases/ffmpeg-4.1.4-amd64-static.tar.xz
Resolving www.johnvansickle.com (www.johnvansickle.com)... 107.180.57.212
Connecting to www.johnvansickle.com (www.johnvansickle.com)|107.180.57.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36323072 (35M) [application/x-xz]
Saving to: ‘ffmpeg-4.1.4-amd64-static.tar.xz’

ffmpeg-4.1.4-amd64- 100%[===================>]  34.64M  34.3MB/s    in 1.0s    

2021-10-23 14:41:05 (34.3 MB/s) - ‘ffmpeg-4.1.4-amd64-static.tar.xz’ saved [36323072/36323072]



In [ ]:
!tar xvf ffmpeg-4.1.4-amd64-static.tar.xz

ffmpeg-4.1.4-amd64-static/
ffmpeg-4.1.4-amd64-static/GPLv3.txt
ffmpeg-4.1.4-amd64-static/manpages/
ffmpeg-4.1.4-amd64-static/manpages/ffmpeg-all.txt
ffmpeg-4.1.4-amd64-static/manpages/ffmpeg-scaler.txt
ffmpeg-4.1.4-amd64-static/manpages/ffmpeg-resampler.txt
ffmpeg-4.1.4-amd64-static/manpages/ffmpeg-filters.txt
ffmpeg-4.1.4-amd64-static/manpages/ffprobe.txt
ffmpeg-4.1.4-amd64-static/manpages/ffmpeg-devices.txt
ffmpeg-4.1.4-amd64-static/manpages/ffmpeg-utils.txt
ffmpeg-4.1.4-amd64-static/manpages/ffmpeg-protocols.txt
ffmpeg-4.1.4-amd64-static/manpages/ffmpeg-codecs.txt
ffmpeg-4.1.4-amd64-static/manpages/ffmpeg-bitstream-filters.txt
ffmpeg-4.1.4-amd64-static/manpages/ffmpeg.txt
ffmpeg-4.1.4-amd64-static/manpages/ffmpeg-formats.txt
ffmpeg-4.1.4-amd64-static/ffprobe
ffmpeg-4.1.4-amd64-static/qt-faststart
ffmpeg-4.1.4-amd64-static/model/
ffmpeg-4.1.4-amd64-static/model/vmaf_v0.6.1.pkl.model
ffmpeg-4.1.4-amd64-static/model/vmaf_4k_v0.6.1.pkl
ffmpeg-4.1.4-amd64-static/model/vmaf_rb_v0.6.3/
ffm

In [ ]:
!zip -r ffmpeg1.zip ffmpeg-4.1.4-amd64-static

  adding: ffmpeg-4.1.4-amd64-static/ (stored 0%)
  adding: ffmpeg-4.1.4-amd64-static/GPLv3.txt (deflated 66%)
  adding: ffmpeg-4.1.4-amd64-static/manpages/ (stored 0%)
  adding: ffmpeg-4.1.4-amd64-static/manpages/ffmpeg-all.txt (deflated 72%)
  adding: ffmpeg-4.1.4-amd64-static/manpages/ffmpeg-scaler.txt (deflated 63%)
  adding: ffmpeg-4.1.4-amd64-static/manpages/ffmpeg-resampler.txt (deflated 66%)
  adding: ffmpeg-4.1.4-amd64-static/manpages/ffmpeg-filters.txt (deflated 72%)
  adding: ffmpeg-4.1.4-amd64-static/manpages/ffprobe.txt (deflated 69%)
  adding: ffmpeg-4.1.4-amd64-static/manpages/ffmpeg-devices.txt (deflated 72%)
  adding: ffmpeg-4.1.4-amd64-static/manpages/ffmpeg-utils.txt (deflated 67%)
  adding: ffmpeg-4.1.4-amd64-static/manpages/ffmpeg-protocols.txt (deflated 68%)
  adding: ffmpeg-4.1.4-amd64-static/manpages/ffmpeg-codecs.txt (deflated 72%)
  adding: ffmpeg-4.1.4-amd64-static/manpages/ffmpeg-bitstream-filters.txt (deflated 66%)
  adding: ffmpeg-4.1.4-amd64-static/manpage

In [ ]:
from utils.file_utils import *
fpath="crawl/oct21_au/classified_new/com/transport-services-for-the-food-beverages-industry.txt"
out=get_multiple_lines(20,15,fpath)
print(out)
print(len(out))
#!ls crawl/oct21_au/classified_new/com

['au.com.cgosystem', 'au.com.cresolutions', 'au.com.mefreightmanagement', 'au.com.mywlogistics', 'au.com.wiseway', 'au.com.keysfreight', 'au.com.camex', 'au.com.whittamtransport', 'au.com.acitgroup', 'au.com.formby', 'au.com.zenith-freight', 'au.net.wbl', 'au.com.kalgin', 'au.com.rapidhaulage', 'au.com.cdbwarehousing']
15


In [ ]:
import json
structure_fname="data_dict.json"
with open(structure_fname) as fopen:
  structure_dict=json.load(fopen)

parent_list=structure_dict["parent_list"]
child_dict=structure_dict["child_dict"]
# for key,children in child_dict.items():
#   print(key)
#   for child0 in children:
#     print(child0)
#     grand_children=child_dict.get(child0[0],[])
#     print(">>> grand_children", grand_children)
#   print("----")
def get_children(child_dict0,cat_id0,level=0):
  for child in child_dict0.get(cat_id0,[]): #node['children']:
    yield child
    if cat_id0==child[0]: continue
    for grandchild in get_children(child_dict0,child[0],level+1):
      yield grandchild
recursive_child_dict={}
for key in child_dict:
  grand_children=list(get_children(child_dict,key))
  recursive_child_dict[key]=grand_children
  print(key,grand_children)

accounting [['accounting-services', 'Accounting & Bookkeeping']]
animal-farming [['seafood-farming-supply', 'Seafood Farming & Supply'], ['animal-transport-handling', 'Animal Transport & Handling'], ['animal-nutrition-health-products', 'Animal Nutrition & Health'], ['poultry-farming', 'Poultry Farming'], ['livestock-farming', 'Livestock Farming'], ['abattoirs', 'Abattoirs'], ['animal-veterinary-services', 'Veterinary Services']]
audit-compliance [['financial-audit-compliance-services', 'Financial Audit Services'], ['fraud-incident-investigation-services', 'Fraud Investigation'], ['food-beverages-audit-compliance-related-services', 'Food Safety & Compliance'], ['asset-property-inspection-verification', 'Asset & Property Inspection'], ['environmental-consulting-audit-services', 'Environmental Consulting'], ['safety-audit-compliance-services', 'Workplace Safety']]
business-communication [['printing-publishing-services', 'Printing & Publishing'], ['translation-interpreting-services', 'Tran

In [ ]:
import re

class element_tag:
  def __init__(self):
    self.inner_html=""
    self.outer_html=""
    self.tag_name=""
    self.open_tag=""
    self.closing_tag=""
    self.class_name=""
    self.classes=[]
    self.children=[]
    self.attrs={}
    self.id=""

# class html_page:
#   def __init__(self,page_fpath):

with open("test.html") as fopen:
  html_content=fopen.read()
html_content=re.sub("(<!--.*?-->)", "", html_content, flags=re.DOTALL) #remove html comments
html_content=re.sub("(<!.+?>)", "", html_content, flags=re.DOTALL) #remove html doc type
html_content=re.sub('(?i)<script[\s\S]+?/script>', "", html_content) #remove script tags
html_content=re.sub('(?i)<style[\s\S]+?/style>', "", html_content) #remove style tags

open_tags=[]
tag_dict={}
tag_obj_dict={}
tag_counter_dict={}
tag_txt_dict={}
tags=list(re.finditer('<([^<>]*?)>', html_content))
tag_id_list=[]
tag_class_dict={}
tag_id_dict={}

for t in tags:
  tag_str,tag_start,tag_end=t.group(0), t.start(), t.end()
  tag_name=re.findall(r'</?(.+?)[\s>]',tag_str)[0].lower()
  tag_count=tag_counter_dict.get(tag_name,0)
  tag_id="%s_%s"%(tag_name,tag_count)
  tag_dict[tag_id]=tag_start,tag_end
  
  last_tag=""
  if open_tags: last_tag=open_tags[-1]
  if tag_str.startswith('</'):
    tag_type="c"
    cur_span=html_content[tag_dict[last_tag][1]:tag_start]
    cur_span_clean=re.sub('<([^<>]*?)>',"", cur_span)
    tag_txt_dict[last_tag]=cur_span_clean
    cur_inner_html=html_content[tag_dict[last_tag][1]:tag_start]
    cur_outer_html=html_content[tag_dict[last_tag][0]:tag_end]
    # last_tag_obj=tag_obj_dict[last_tag]
    # last_tag_obj.inner_html=cur_inner_html
    # last_tag_obj.outer_html=cur_outer_html
    # last_tag_obj.closing_tag=tag_str
    # tag_obj_dict[last_tag]=last_tag_obj
    tag_obj_dict[last_tag].inner_html=cur_inner_html
    tag_obj_dict[last_tag].outer_html=cur_outer_html
    tag_obj_dict[last_tag].closing_tag=tag_str

    #full_span_dict[last_tag]=cur_span
    open_tags=open_tags[:-1]
    cur_attrs={}
  elif tag_str.endswith('/>') or tag_str.startswith('<meta') or tag_str.startswith('<img'):
    tag_type='s'
    #children_dict[last_tag]=self.children_dict.get(last_tag,[])+[tag_id]
    tag_counter_dict[tag_name]=tag_count+1
    #attrs_dict[tag_id]
    cur_attrs=dict(iter(re.findall('(\w+?)="(.+?)"',tag_str)))
    #actual_id=
  else:
    tag_type='o'
    #if tag_name==main_tag_name_criteria: self.main_keys.append(tag_id)
    open_tags.append(tag_id)
    #self.children_dict[last_tag]=self.children_dict.get(last_tag,[])+[tag_id]
    tag_counter_dict[tag_name]=tag_count+1
    cur_attrs=dict(iter(re.findall('(\w+?)="(.+?)"',tag_str)))
    class_name=""
    original_id=""
    for key,val in cur_attrs.items():
      if key.lower()=="class": class_name=val
      if key.lower()=="id": original_id=val
    class_list=class_name.split()
    element_obj=element_tag()
    element_obj.id=original_id
    element_obj.class_name=class_name
    element_obj.class_list=class_list
    element_obj.tag_name=tag_name
    element_obj.open_tag=tag_str

    tag_obj_dict[tag_id]=element_obj
    if original_id!="": tag_id_dict[original_id]=tag_id
    for class_item in class_list:
      tag_class_dict[class_item]=tag_class_dict.get(class_item,[])+[tag_id]
    tag_id_list.append(tag_id)
    #cur_class=cur_attrs.get("class","")
    #self.attrs_dict[tag_id]=dict(iter(re.findall('(\w+?)="(.+?)"',tag_str))) 
    #print(tag_id, tag_type,cur_attrs, class_list, "id:", original_id)

# for k,v in tag_obj_dict.items():
#   print(k,v.open_tag, v.closing_tag)

cur_list=tag_class_dict.get("modal-body",[])
for tag_id0 in cur_list:
  corr_obj=tag_obj_dict[tag_id0]
  print(tag_id0, corr_obj.open_tag, corr_obj.closing_tag)
# scripts=re.findall('(?i)<script[\s\S]+?/script>', html_content)
# styles=re.findall('(?i)<style[\s\S]+?/style>', html_content)
# for sc in styles:
#   print(sc)
#html_content=re.findall()
#print(html_content[:100])
# self.main_keys=[]
# tags=list(re.finditer('<([^<>]*?)>', xml_content))
# open_tags=[]
# tag_dict={}
# self.tag_txt_dict={}
# tag_counter_dict={}
# self.children_dict={}
# self.attrs_dict={}
# self.full_span_dict={}
# #sent_tokens=[]
# for t in tags:

# tag_str,tag_start,tag_end=t.group(0), t.start(), t.end()
# tag_name=re.findall(r'</?(.+?)[\s>]',tag_str)[0].lower()
# tag_count=tag_counter_dict.get(tag_name,0)
# tag_id="%s_%s"%(tag_name,tag_count)
# tag_dict[tag_id]=tag_start,tag_end


# last_tag=""
# if open_tags: last_tag=open_tags[-1]
# if tag_str.startswith('</'):
# tag_type="c"

# cur_span=xml_content[tag_dict[last_tag][1]:tag_start]
# cur_span_clean=re.sub('<([^<>]*?)>',"", cur_span)
# self.tag_txt_dict[last_tag]=cur_span_clean
# self.full_span_dict[last_tag]=cur_span

# open_tags=open_tags[:-1]
# elif tag_str.endswith('/>') or tag_str.startswith('<meta') or tag_str.startswith('<img'):
# tag_type='s'
# self.children_dict[last_tag]=self.children_dict.get(last_tag,[])+[tag_id]
# tag_counter_dict[tag_name]=tag_count+1
# self.attrs_dict[tag_id]=dict(iter(re.findall('(\w+?)="(.+?)"',tag_str)))

# else:
# tag_type='o'
# if tag_name==main_tag_name_criteria: self.main_keys.append(tag_id)
# open_tags.append(tag_id)
# self.children_dict[last_tag]=self.children_dict.get(last_tag,[])+[tag_id]
# tag_counter_dict[tag_name]=tag_count+1
# self.attrs_dict[tag_id]=dict(iter(re.findall('(\w+?)="(.+?)"',tag_str)))       

div_214 <div class="modal-body"> </div>
div_224 <div class="modal-body"> 
div_236 <div class="modal-body"> 
div_250 <div class="modal-body"> </div>
div_259 <div class="modal-body"> </div>
div_267 <div class="modal-body"> </div>
div_273 <div class="modal-body"> </div>


In [ ]:
class element_tag:
  def __init__(self):
    self.inner_html=""
    self.outer_html=""
    self.tag_name=""
    self.open_tag=""
    self.closing_tag=""
    self.class_name=""
    self.classes=[]
    self.children=[]
    self.attrs={}
    self.id=""

class html_page:
  def __init__(self,page_fpath):
    with open(page_fpath) as fopen:
      self.html_content=fopen.read()
    self.original_html_content=str(self.html_content)
    self.html_content=re.sub("(<!--.*?-->)", "", self.html_content, flags=re.DOTALL) #remove html comments
    self.html_content=re.sub("(<!.+?>)", "", self.html_content, flags=re.DOTALL) #remove html doc type
    self.html_content=re.sub('(?i)<script[\s\S]+?/script>', "", self.html_content) #remove script tags
    self.html_content=re.sub('(?i)<style[\s\S]+?/style>', "", self.html_content) #remove style tags

    open_tags=[] #keep track of which tags are open
    tag_counter_dict={}

    self.tag_dict={} #span (start loc, end loc) - key is the automatic tag id
    self.tag_obj_dict={}    #element obj 
    self.tag_id_list=[] # list of automatic tag ids
    self.tag_class_dict={} #list of automatic tag ids for each class
    self.tag_id_dict={} #the corresponding automatic tag id for each original id in the element <tag id="original">
    self.child_dict={} #which tag has which children - key is automatic tag id, children are list of automatic tag ids
    self.tag_txt_dict={} # just get the clean text of each tag/element
    self.title=""
    self.description=""

    tags=list(re.finditer('<([^<>]*?)>', self.html_content)) #then get each individual tag
    for t in tags:
      tag_str,tag_start,tag_end=t.group(0), t.start(), t.end()
      tag_name=re.findall(r'</?(.+?)[\s>]',tag_str)[0].lower()
      tag_count=tag_counter_dict.get(tag_name,0)
      tag_id="%s_%s"%(tag_name,tag_count) #automatically generated ID for each tag
      self.tag_dict[tag_id]=tag_start,tag_end
      
      last_tag=""
      if open_tags: last_tag=open_tags[-1]
      if tag_str.startswith('</'):
        tag_type="c"
        cur_span=self.html_content[self.tag_dict[last_tag][1]:tag_start]
        #if tag_str=="": print("??????????????", cur_span)
        #cur_span_clean=re.sub('<([^<>]*?)>',"", cur_span)
        self.tag_txt_dict[last_tag]=cur_span
        cur_inner_html=self.html_content[self.tag_dict[last_tag][1]:tag_start]
        cur_outer_html=self.html_content[self.tag_dict[last_tag][0]:tag_end]
        self.tag_obj_dict[last_tag].inner_html=cur_inner_html 
        self.tag_obj_dict[last_tag].outer_html=cur_outer_html
        
        self.tag_obj_dict[last_tag].closing_tag=tag_str
        #print("????? ",self.tag_obj_dict[last_tag].id, self.tag_obj_dict[last_tag].closing_tag)

        #full_span_dict[last_tag]=cur_span
        open_tags=open_tags[:-1]
        #cur_attrs={}
      elif tag_str.endswith('/>') or tag_str.lower().startswith('<meta') or tag_str.lower().startswith('<img') or tag_str.lower().startswith('<link'):
        tag_type='s' #stand alone tag
        #children_dict[last_tag]=self.children_dict.get(last_tag,[])+[tag_id]
        tag_counter_dict[tag_name]=tag_count+1
        #attrs_dict[tag_id]
        cur_attrs=dict(iter(re.findall('(\w+?)="(.+?)"',tag_str)))
        class_name=""
        original_id=""
        for key,val in cur_attrs.items():
          if key.lower()=="class": class_name=val
          if key.lower()=="id": original_id=val   
        class_list=class_name.split()     
        element_obj=element_tag()
        element_obj.id=original_id
        element_obj.class_name=class_name
        element_obj.class_list=class_list
        element_obj.tag_name=tag_name
        element_obj.open_tag=tag_str
        element_obj.attrs=cur_attrs
        self.tag_obj_dict[tag_id]=element_obj
                
        #actual_id=
      else:
        tag_type='o'
        #if tag_name==main_tag_name_criteria: self.main_keys.append(tag_id)
        open_tags.append(tag_id)
        #self.children_dict[last_tag]=self.children_dict.get(last_tag,[])+[tag_id]
        tag_counter_dict[tag_name]=tag_count+1
        cur_attrs=dict(iter(re.findall('(\w+?)="(.+?)"',tag_str)))
        class_name=""
        original_id=""
        for key,val in cur_attrs.items():
          if key.lower()=="class": class_name=val
          if key.lower()=="id": original_id=val
        class_list=class_name.split()
        element_obj=element_tag()
        element_obj.id=original_id
        element_obj.class_name=class_name
        element_obj.class_list=class_list
        element_obj.tag_name=tag_name
        element_obj.open_tag=tag_str
        element_obj.attrs=cur_attrs

        self.tag_obj_dict[tag_id]=element_obj
        if original_id!="": self.tag_id_dict[original_id]=tag_id
        for class_item in class_list:
          self.tag_class_dict[class_item]=self.tag_class_dict.get(class_item,[])+[tag_id]
        self.tag_id_list.append(tag_id)
    #cur_class=cur_attrs.get("class","")
    #self.attrs_dict[tag_id]=dict(iter(re.findall('(\w+?)="(.+?)"',tag_str))) 
    #print(tag_id, tag_type,cur_attrs, class_list, "id:", original_id)
    print(open_tags)
  def get_elements_by_class(self,class_name0):
    cur_element_auto_ids=self.tag_class_dict.get(class_name0,[])
    return [self.tag_obj_dict[v] for v in cur_element_auto_ids]
  def get_element_by_id(self,original_id0):
    cur_element_auto_id=tag_id_dict.get(original_id0,"")
    if cur_element_auto_id=="": return
    return self.tag_obj_dict[cur_element_auto_id]
  def replace_by_id(self,original_id0,new_inner_html):
    cur_el_obj=self.get_element_by_id(original_id0)
    if cur_el_obj==None: return
    original_outer_html=cur_el_obj.outer_html
    new_outer_html=cur_el_obj.open_tag+new_inner_html+cur_el_obj.closing_tag
    return [original_outer_html,new_outer_html]
  def replace_by_class(self,class_name0,new_inner_html):
    cur_element_auto_ids=self.tag_class_dict.get(class_name0,[])
    replacement_pairs=[]
    for auto_tag_id in cur_element_auto_ids:
      cur_el_obj1=self.tag_obj_dict[auto_tag_id]
      #print("??????", auto_tag_id,cur_el_obj1.open_tag,cur_el_obj1.closing_tag)
      original_outer_html=cur_el_obj1.outer_html
      #print("@@@@",cur_el_obj1.inner_html)
      new_outer_html=cur_el_obj1.open_tag+new_inner_html+cur_el_obj1.closing_tag
      replacement_pairs.append([original_outer_html,new_outer_html]) 
    return replacement_pairs
  def replace_by_dict(self,repl_dict0):
    original_html_copy=str(self.original_html_content)
    all_replacements=[]
    for key,val in repl_dict0.items():
      if key.startswith("#"): #we use jquery notation # to indicate element ID
        cur_id_pair=self.replace_by_id(key[1:],val)
        all_replacements.append(cur_id_pair)
      if key.startswith("."): #we use jquery notation . to indicate element classname
        cur_class_pairs=self.replace_by_class(key[1:],val)
        all_replacements.extend(cur_class_pairs) 
    for a,b in all_replacements:
      original_html_copy=original_html_copy.replace(a,b)
    return original_html_copy 
  def update_replace(self, repl_dict0, title, description="",keywords=""):
    out_html=self.replace_by_dict(repl_dict0)  
    new_title="<title>"+title+"</title>" 
    new_description='<meta name="description" content="%s">'%description #"<title>"+title+"</title>" 
    out_html=re.sub('(?i)<title.+?/title>',new_title,out_html)
    out_html=re.sub('(?i)<meta name="description".+?>',new_description,out_html)
    return out_html 



    #return [self.tag_obj_dict[v] for v in cur_element_auto_ids]
    
fname="test.html"
page_obj=html_page(fname)

repl_dict={}
repl_dict["#test1"]="testing testing"
repl_dict[".clip"]="clipping here"
#out_html=page_obj.replace_by_dict(repl_dict)

out_html=page_obj.update_replace(repl_dict,"our new title","and a description as well")

fopen=open("test-html-out.html","w")
fopen.write(out_html)
fopen.close()

# for k,v in tag_obj_dict.items():
#   print(k,v.open_tag, v.closing_tag)
# for a in page_obj.get_elements_by_class("modal-body"):
#   print(a.inner_html[:20],a.open_tag, a.closing_tag)

# for a in page_obj.tag_id_list:
#   cur_obj=page_obj.tag_obj_dict[a]
#   cl=cur_obj.closing_tag.strip()
#   #if cl=="": print(a, cur_obj.open_tag,">>>", cur_obj.closing_tag)


# cur_el=page_obj.get_element_by_id("test1")
# #print(cur_el.outer_html)
# out=page_obj.replace_by_id("test1","whatever, anything")
# #print(out)

# out2=page_obj.replace_by_class("modal-body","whatever, anything")
# for o2 in out2:
#   print(">>>>", o2)
# cur_list=tag_class_dict.get("modal-body",[])
# for tag_id0 in cur_list:
#   corr_obj=tag_obj_dict[tag_id0]
#   print(tag_id0, corr_obj.open_tag, corr_obj.closing_tag)


['html_0', 'body_0', 'div_189', 'div_210', 'div_211', 'div_220', 'div_221', 'div_222', 'div_224', 'div_232', 'div_233', 'div_234', 'div_236', 'div_237']


In [ ]:
from utils.template_utils import *

fname="test.html"
page_obj=html_page(fname)

repl_dict={}
repl_dict["#test1"]="testing testing"
repl_dict[".clip"]="clipping here"
#out_html=page_obj.replace_by_dict(repl_dict)

out_html=page_obj.update_replace(repl_dict,"our new title","and a description as well")

fopen=open("test-html-out1.html","w")
fopen.write(out_html)
fopen.close()


['html_0', 'body_0', 'div_189', 'div_210', 'div_211', 'div_220', 'div_221', 'div_222', 'div_224', 'div_232', 'div_233', 'div_234', 'div_236', 'div_237']


In [ ]:
from utils.extraction_utils import *
test_url='https://anteristech.com/news/ASX_SMID_Conference_March21'
page_obj=web_page(test_url)
content=get_content(test_url)
print(content[:100])

https://anteristech.com/news/ASX_SMID_Conference_March21<br>{"title": "Anteristech - ASX Small Mid C


In [ ]:
for s0 in page_obj.segs[:10]:
  print(s0)

https://anteristech.com/news/ASX_SMID_Conference_March21
{"title": "Anteristech - ASX Small Mid Conference - Anteris Technologies", "description": "Anteris Technologies ASX SMID Conference on demand video"}
Anteristech - ASX Small Mid Conference - Anteris Technologies
Menu
Home Right
News & Events Right
ASX Small Mid Conference - Anteris Technologies
ASX Small Mid Conference - Anteris Technologies
Share
See the Anteris Technologies ASX Small Mid Conference on demand video below.
